# Spectral Deconvolution for PDC Enzyme Assays

**Author:** Dan  
**Date:** January 2026

This notebook analyzes spectrophotometric data from Pyruvate Dehydrogenase Complex (PDC) enzyme assays using spectral deconvolution to accurately quantify NADH and pyruvate concentrations.

## Workflow Overview

1. **Setup & Data Loading** - Mount drive, import libraries, locate data files
2. **Load Standard Spectra** - Parse .WAV files containing NADH and pyruvate standards
3. **Build Extinction Coefficient Standards** - Calculate and validate extinction coefficients
4. **Core Analysis Functions** - Spectral deconvolution with optional fixed concentrations
5. **Load Kinetic Data** - Import time course experimental data
6. **PDC Time Course Analysis** - Automated processing with blank correction
7. **Example Analysis** - Process and visualize CELL_1 data


## Setup & Data Loading

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

Mounted at /content/drive
Google Drive mounted successfully.


In [2]:
import os
file_path = '/content/drive/MyDrive/Research/PDC+ADH+FDH assay data Evelyn 2025/Spectrum files from Agilent spec'

print(f"Current working directory: {os.getcwd()}")
os.chdir(file_path)
print(f"New working directory: {os.getcwd()}")

Current working directory: /content
New working directory: /content/drive/MyDrive/Research/PDC+ADH+FDH assay data Evelyn 2025/Spectrum files from Agilent spec


In [3]:
import glob
import os

# Find all .WAV files recursively relative to the current working directory
wav_files = glob.glob('*.WAV', recursive=True)

# The current working directory is already set, so relative paths are sufficient.

print(f"Found {len(wav_files)} .wav files (relative paths):")
for file_path in wav_files:
    print(file_path)

Found 9 .wav files (relative paths):
0_05MM NADH SPECTRUM.WAV
0_15MM NADH SPECTRUM.WAV
0_25MM NADH SPECTRUM.WAV
1MM NADH SPECTRUM.WAV
1MM PYR SPECTRUM.WAV
10MM PYR SPECTRUM.WAV
100MM PYR SPECTRUM.WAV
50MM PYR SPECTRUM.WAV
200mM TRIS SPECTRUM.WAV


---
<a id='standards'></a>
## Load Standard Spectra

### Parse Standard Spectrum Files (.WAV)

Load and compile NADH and pyruvate standard spectra from Agilent spectrophotometer files.

In [4]:
import re
import pandas as pd
import numpy as np

all_spectra_df = []

for file_name in wav_files:
    # Open the file and read lines to extract wavelength info
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Extract start and end wavelengths from the 8th line (index 7)
    wavelength_info = lines[7].strip().split(',')
    start_wavelength = float(wavelength_info[0])
    end_wavelength = float(wavelength_info[1])

    # Load the data from row 9 onwards (skipping the first 8 rows)
    # Select only the first column for Absorbance
    raw_data = pd.read_csv(file_name, skiprows=8, header=None)
    temp_df = pd.DataFrame(raw_data.iloc[:, 0]) # Extract only the first column as Absorbance
    temp_df.columns = ['Absorbance']

    # Generate 'Wavelength' column
    num_data_points = temp_df.shape[0]
    wavelength_column = np.linspace(start_wavelength, end_wavelength, num_data_points)

    # Insert 'Wavelength' column as the first column
    temp_df.insert(0, 'Wavelength', wavelength_column)

    # Add 'Compound' column extracted from filename
    # Extract compound name from the original filename (without path) for clarity
    original_file_name = os.path.basename(file_name)
    if 'TRIS' in original_file_name.upper():
        temp_df['Compound'] = 'TRIS'
    elif 'NADH' in original_file_name.upper():
        temp_df['Compound'] = 'NADH'
    elif 'PYR' in original_file_name.upper():
        temp_df['Compound'] = 'PYR'
    else:
        temp_df['Compound'] = 'Unknown'

    # Add 'File_Name' column (using the relative path now)
    temp_df['File_Name'] = file_name

    # Add 'Expected_mM' column by extracting concentration from file_name
    expected_mm_value = np.nan
    match = re.search(r'([0-9_.]+)MM', original_file_name.upper())
    if match:
        concentration_str = match.group(1).replace('_', '.')
        try:
            expected_mm_value = float(concentration_str)
        except ValueError:
            pass # Keep as np.nan if conversion fails
    temp_df['Expected_mM'] = expected_mm_value

    # Append to the list
    all_spectra_df.append(temp_df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(all_spectra_df, ignore_index=True)

# Display the first few rows and information about the DataFrame
display(merged_df.head())
display(merged_df.info())

,Wavelength,Absorbance,Compound,File_Name,Expected_mM
0,190.0,2.548025,NADH,0_05MM NADH SPECTRUM.WAV,0.05
1,191.0,2.592308,NADH,0_05MM NADH SPECTRUM.WAV,0.05
2,192.0,2.776916,NADH,0_05MM NADH SPECTRUM.WAV,0.05
3,193.0,2.935247,NADH,0_05MM NADH SPECTRUM.WAV,0.05
4,194.0,2.876520,NADH,0_05MM NADH SPECTRUM.WAV,0.05


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8199 entries, 0 to 8198
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Wavelength   8199 non-null   float64
 1   Absorbance   8199 non-null   float64
 2   Compound     8199 non-null   object 
 3   File_Name    8199 non-null   object 
 4   Expected_mM  8199 non-null   float64
dtypes: float64(3), object(2)
memory usage: 320.4+ KB


None

## Plot the spectra from each data file

In [5]:
import plotly.express as px

fig = px.line(
    merged_df,
    x='Wavelength',
    y='Absorbance',
    #line_dash ='Compound',
    color = 'Expected_mM',
    title='Combined Spectral Data for All WAV Files',
    facet_row = 'Compound',

)

fig.update_layout(
    xaxis_title='Wavelength (nm)',
    yaxis_title='Absorbance'
)

fig.show()

## Plot Concentration vs. Absorbance
Create two separate plots: one for NADH showing 'Expected_mM' vs. 'Absorbance' at 340 nm, and another for Pyruvate showing 'Expected_mM' vs. 'Absorbance' at 320 nm. Ensure plots have appropriate labels, titles, and markers for clarity.


In [6]:
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as smf

TARGET_WAVELENGTH_NADH = 340

# 1. Filter the merged_df for NADH data at 340 nm and Absorbance <= 2.5
nadh_plot_df = merged_df[
    (merged_df['Compound'] == 'NADH') & # Use the Compound column for filtering
    merged_df['Expected_mM'].notna() &
    (merged_df['Wavelength'] == TARGET_WAVELENGTH_NADH) &
    (merged_df['Absorbance'] <= 2.5) # Filter out absorbance > 2.5
].drop_duplicates(subset=['File_Name']).copy()

# Display the filtered data for verification
print("Filtered NADH Plotting Data (Absorbance <= 2.5):")
display(nadh_plot_df)

# Perform linear regression forced through zero
# Using statsmodels to fit y = m*x (no intercept)
if not nadh_plot_df.empty:
    model_nadh = smf.ols('Absorbance ~ 0 + Expected_mM', data=nadh_plot_df).fit()
    slope_nadh = model_nadh.params['Expected_mM']
    r_squared_nadh = model_nadh.rsquared

    # Generate data for the regression line
    x_reg_nadh = np.linspace(0, nadh_plot_df['Expected_mM'].max() * 1.1, 100)
    y_reg_nadh = slope_nadh * x_reg_nadh

    # Create the plot using plotly.graph_objects
    fig_nadh = go.Figure()

    # Add scatter plot of actual data
    fig_nadh.add_trace(go.Scatter(
        x=nadh_plot_df['Expected_mM'],
        y=nadh_plot_df['Absorbance'],
        mode='markers',
        name='Actual Data'
    ))

    # Add regression line
    fig_nadh.add_trace(go.Scatter(
        x=x_reg_nadh,
        y=y_reg_nadh,
        mode='lines',
        name=f'Linear Regression (y={slope_nadh:.4f}x)',
        line=dict(color='red', dash='dash')
    ))

    # Update layout and add annotations
    fig_nadh.update_layout(
        title=f'NADH Expected Concentration vs. Absorbance at {TARGET_WAVELENGTH_NADH} nm (Absorbance <= 2.5)',
        xaxis_title='Expected Concentration (mM)',
        yaxis_title='Absorbance',
        annotations=[
            dict(
                x=0.05, y=0.95,
                xref='paper', yref='paper',
                text=f'Slope: {slope_nadh:.4f}<br>R-squared: {r_squared_nadh:.4f}',
                showarrow=False,
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="black",
                borderwidth=1,
                xanchor='left', yanchor='top'
            )
        ]
    )

    fig_nadh.show()
else:
    print(f"No NADH data found for plotting after filtering absorbance > 2.5 at {TARGET_WAVELENGTH_NADH} nm.")

Filtered NADH Plotting Data (Absorbance <= 2.5):


,Wavelength,Absorbance,Compound,File_Name,Expected_mM
150,340.0,0.269806,NADH,0_05MM NADH SPECTRUM.WAV,0.05
1061,340.0,0.833405,NADH,0_15MM NADH SPECTRUM.WAV,0.15
1972,340.0,1.387911,NADH,0_25MM NADH SPECTRUM.WAV,0.25


## Adjust the NADH concentration
Adjust the NADH 'Expected_mM' values by a scaling factor derived from the previous plot's slope (5.5483) and the theoretical extinction coefficient (6.22) to create a new 'Concentration_mM' column. Replot the NADH concentration vs. Absorbance at 340 nm using this new column to confirm that the linear regression now yields a slope of 6.22.

In [7]:
import plotly.graph_objects as go
import statsmodels.formula.api as smf
import pandas as pd # Ensure pandas is imported as it's used directly on merged_df
import numpy as np # Ensure numpy is imported for np.linspace

EXTINCTION_COEFFICIENT_NADH = 6.22

# Define the theoretical slope (extinction coefficient)
theoretical_slope = EXTINCTION_COEFFICIENT_NADH

# Get the previously calculated experimental slope for NADH from the kernel state
experimental_slope = slope_nadh # From previous execution

# Calculate the adjustment factor for plotting: actual_concentration = Expected_mM * (experimental_slope / theoretical_slope)
scaling_factor_for_concentration = experimental_slope / theoretical_slope

print(f"Theoretical Slope (Extinction Coefficient): {theoretical_slope:.4f}")
print(f"Experimental Slope from previous plot: {experimental_slope:.4f}")
print(f"Scaling Factor for Concentration Adjustment: {scaling_factor_for_concentration:.4f}")

# Initialize the 'Concentration_mM' column in merged_df if it doesn't exist
if 'Concentration_mM' not in merged_df.columns:
    merged_df['Concentration_mM'] = np.nan

# Apply the concentration adjustment to 'merged_df' for NADH entries across all wavelengths
# This ensures 'Concentration_mM' is available for NADH data at all wavelengths in merged_df
nadh_rows_mask = (merged_df['Compound'] == 'NADH') & merged_df['Expected_mM'].notna()
merged_df.loc[nadh_rows_mask, 'Concentration_mM'] = merged_df.loc[nadh_rows_mask, 'Expected_mM'] * scaling_factor_for_concentration

# Now, filter data specifically for the plot (340nm and Absorbance <= 2.5)
adjusted_nadh_plot_df = merged_df[
    (merged_df['Compound'] == 'NADH') &
    merged_df['Concentration_mM'].notna() & # Use the newly created Concentration_mM
    (merged_df['Wavelength'] == TARGET_WAVELENGTH_NADH) &
    (merged_df['Absorbance'] <= 2.5)
].drop_duplicates(subset=['File_Name']).copy()


print("\nAdjusted NADH Plotting Data (Absorbance <= 2.5, with new Concentration_mM for plot):")
display(adjusted_nadh_plot_df)

# Perform linear regression forced through zero with the new 'Concentration_mM'
if not adjusted_nadh_plot_df.empty:
    model_nadh_adjusted = smf.ols('Absorbance ~ 0 + Concentration_mM', data=adjusted_nadh_plot_df).fit()
    new_slope_nadh = model_nadh_adjusted.params['Concentration_mM']
    new_r_squared_nadh = model_nadh_adjusted.rsquared

    # Generate data for the new regression line
    x_reg_nadh_adjusted = np.linspace(0, adjusted_nadh_plot_df['Concentration_mM'].max() * 1.1, 100)
    y_reg_nadh_adjusted = new_slope_nadh * x_reg_nadh_adjusted

    # Create the plot using plotly.graph_objects
    fig_nadh_adjusted = go.Figure()

    # Add scatter plot of actual data (using adjusted concentrations)
    fig_nadh_adjusted.add_trace(go.Scatter(
        x=adjusted_nadh_plot_df['Concentration_mM'],
        y=adjusted_nadh_plot_df['Absorbance'],
        mode='markers',
        name='Actual Data (Adjusted Concentration)'
    ))

    # Add regression line
    fig_nadh_adjusted.add_trace(go.Scatter(
        x=x_reg_nadh_adjusted,
        y=y_reg_nadh_adjusted,
        mode='lines',
        name=f'Linear Regression (y={new_slope_nadh:.4f}x)',
        line=dict(color='red', dash='dash')
    ))

    # Update layout and add annotations
    fig_nadh_adjusted.update_layout(
        title=f'NADH Adjusted Concentration vs. Absorbance at {TARGET_WAVELENGTH_NADH} nm (Absorbance <= 2.5)',
        xaxis_title='Adjusted Concentration (mM)',
        yaxis_title='Absorbance',
        annotations=[
            dict(
                x=0.05, y=0.95,
                xref='paper', yref='paper',
                text=f'Slope: {new_slope_nadh:.4f}<br>R-squared: {new_r_squared_nadh:.4f}',
                showarrow=False,
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="black",
                borderwidth=1,
                xanchor='left', yanchor='top'
            )
        ]
    )

    fig_nadh_adjusted.show()
else:
    print(f"No NADH data found for plotting after filtering absorbance > 2.5 at {TARGET_WAVELENGTH_NADH} nm.")

Theoretical Slope (Extinction Coefficient): 6.2200
Experimental Slope from previous plot: 5.5483
Scaling Factor for Concentration Adjustment: 0.8920

Adjusted NADH Plotting Data (Absorbance <= 2.5, with new Concentration_mM for plot):


,Wavelength,Absorbance,Compound,File_Name,Expected_mM,Concentration_mM
150,340.0,0.269806,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601
1061,340.0,0.833405,NADH,0_15MM NADH SPECTRUM.WAV,0.15,0.133802
1972,340.0,1.387911,NADH,0_25MM NADH SPECTRUM.WAV,0.25,0.223004


## Adjust the NADH concentration based on the Abs340 data, convert to a wide-format table and export as a csv file

In [8]:
import pandas as pd

# Filter merged_df for NADH data with calculated Concentration_mM
nadh_data_for_export = merged_df[
    (merged_df['Compound'] == 'NADH') &
    merged_df['Concentration_mM'].notna()
].copy()

# 1. Create a new DataFrame wide_nadh_df by pivoting the nadh_data_for_export.
#    Use 'Wavelength' as the index, 'Concentration_mM' for the new column names, and 'Absorbance' as the values.
wide_nadh_df = nadh_data_for_export.pivot_table(index='Wavelength', columns='Concentration_mM', values='Absorbance')

# 2. Reset the index of wide_nadh_df so 'Wavelength' becomes a regular column.
wide_nadh_df = wide_nadh_df.reset_index()

# 3. Optionally, rename the columns for better readability.
#    The column names are currently float values from 'Concentration_mM'. Convert them to string and add 'NADH_' prefix.
#    The first column 'Wavelength' should be kept as is.

# Get current column names (which are numbers or 'Wavelength')
current_columns = wide_nadh_df.columns.tolist()

# Create new column names, preserving 'Wavelength' and formatting others
new_columns = ['Wavelength']
for col in current_columns[1:]:
    if isinstance(col, (float, int)):
        new_columns.append(f'NADH_{col:.4f}mM_Absorbance') # Format to 4 decimal places for consistency
    else:
        new_columns.append(col)

wide_nadh_df.columns = new_columns

# Display the head of the wide DataFrame
print("Wide NADH DataFrame Head:")
display(wide_nadh_df.head())

# 4. Save the wide_nadh_df DataFrame to a CSV file named 'Adjusted_NADH_Concentration_Absorbance.csv'.
#    Ensure the index is not written to the CSV file.
output_filename_adjusted_nadh = 'Adjusted_NADH_Concentration_Absorbance.csv'
wide_nadh_df.to_csv(output_filename_adjusted_nadh, index=False)
print(f"\nWide NADH DataFrame saved to {output_filename_adjusted_nadh}")

Wide NADH DataFrame Head:


,Wavelength,NADH_0.0446mM_Absorbance,NADH_0.1338mM_Absorbance,NADH_0.2230mM_Absorbance,NADH_0.8920mM_Absorbance
0,190.0,2.548025,2.903203,3.171649,4.000000
1,191.0,2.592308,3.095706,3.095250,3.530418
2,192.0,2.776916,3.389744,3.623547,4.000000
3,193.0,2.935247,3.487925,3.778119,3.973017
4,194.0,2.876520,3.449419,3.555676,3.722551



Wide NADH DataFrame saved to Adjusted_NADH_Concentration_Absorbance.csv


## Plot Pyruvate Expected Concentration vs. Absorbance at 320 nm
Filter the `merged_df` for Pyruvate data at 320 nm and plot 'Absorbance' against 'Expected_mM'. The plot should include a linear regression forced through zero, along with its slope and R-squared value, similar to the NADH plot.


In [9]:
import plotly.graph_objects as go
import statsmodels.formula.api as smf

TARGET_WAVELENGTH_PYR = 320

# 1. Filter the merged_df for Pyruvate data at 320 nm and 'Expected_mM' is not NaN
pyr_plot_df = merged_df[
    (merged_df['Compound'] == 'PYR') &
    merged_df['Expected_mM'].notna() &
    (merged_df['Wavelength'] == TARGET_WAVELENGTH_PYR)
].drop_duplicates(subset=['File_Name']).copy()

# Display the filtered data for verification
print("Filtered Pyruvate Plotting Data:")
display(pyr_plot_df)

# 2. Perform linear regression forced through zero
# Using statsmodels to fit y = m*x (no intercept)
if not pyr_plot_df.empty:
    model_pyr = smf.ols('Absorbance ~ 0 + Expected_mM', data=pyr_plot_df).fit()
    slope_pyr = model_pyr.params['Expected_mM']
    r_squared_pyr = model_pyr.rsquared

    # 3. Generate data for the regression line
    x_reg_pyr = np.linspace(0, pyr_plot_df['Expected_mM'].max() * 1.1, 100)
    y_reg_pyr = slope_pyr * x_reg_pyr

    # 4. Create the scatter plot using plotly.graph_objects
    fig_pyr = go.Figure()

    # Add scatter plot of actual data
    fig_pyr.add_trace(go.Scatter(
        x=pyr_plot_df['Expected_mM'],
        y=pyr_plot_df['Absorbance'],
        mode='markers',
        name='Actual Data',
        marker=dict(color='blue')
    ))

    # 5. Add regression line
    fig_pyr.add_trace(go.Scatter(
        x=x_reg_pyr,
        y=y_reg_pyr,
        mode='lines',
        name=f'Linear Regression (y={slope_pyr:.4f}x)',
        line=dict(color='orange', dash='dash')
    ))

    # 6. Set the plot's title, x-axis label, and y-axis label
    fig_pyr.update_layout(
        title=f'Pyruvate Expected Concentration vs. Absorbance at {TARGET_WAVELENGTH_PYR} nm',
        xaxis_title='Expected Concentration (mM)',
        yaxis_title='Absorbance'
    )

    # 7. Add an annotation displaying the calculated slope and R-squared value
    fig_pyr.add_annotation(
        x=0.05, y=0.95,
        xref='paper', yref='paper',
        text=f'Slope: {slope_pyr:.4f}<br>R-squared: {r_squared_pyr:.4f}',
        showarrow=False,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        xanchor='left', yanchor='top'
    )

    # 8. Display the plot
    fig_pyr.show()
else:
    print(f"No Pyruvate data found for plotting at {TARGET_WAVELENGTH_PYR} nm.")

Filtered Pyruvate Plotting Data:


,Wavelength,Absorbance,Compound,File_Name,Expected_mM,Concentration_mM
3774,320.0,0.010163,PYR,1MM PYR SPECTRUM.WAV,1.0,NaN
4685,320.0,0.194522,PYR,10MM PYR SPECTRUM.WAV,10.0,NaN
5596,320.0,2.029487,PYR,100MM PYR SPECTRUM.WAV,100.0,NaN
6507,320.0,0.966928,PYR,50MM PYR SPECTRUM.WAV,50.0,NaN


In [10]:
import pandas as pd

# Create 'Concentration_mM' column for Pyruvate by copying 'Expected_mM' values
# This should apply to all Pyruvate rows in merged_df
pyruvate_rows_mask = (merged_df['Compound'] == 'PYR') & merged_df['Expected_mM'].notna()
merged_df.loc[pyruvate_rows_mask, 'Concentration_mM'] = merged_df.loc[pyruvate_rows_mask, 'Expected_mM']

# Filter merged_df for Pyruvate data with calculated Concentration_mM for export
pyr_data_for_export = merged_df[
    (merged_df['Compound'] == 'PYR') &
    merged_df['Concentration_mM'].notna()
].copy()

# 1. Create a new DataFrame wide_pyr_df by pivoting the pyr_data_for_export.
#    Use 'Wavelength' as the index, 'Concentration_mM' for the new column names, and 'Absorbance' as the values.
wide_pyr_df = pyr_data_for_export.pivot_table(index='Wavelength', columns='Concentration_mM', values='Absorbance')

# 2. Reset the index of wide_pyr_df so 'Wavelength' becomes a regular column.
wide_pyr_df = wide_pyr_df.reset_index()

# 3. Rename the columns for better readability.
#    The column names are currently float values from 'Concentration_mM'. Convert them to string and add 'PYR_' prefix.
#    The first column 'Wavelength' should be kept as is.

# Get current column names (which are numbers or 'Wavelength')
current_columns_pyr = wide_pyr_df.columns.tolist()

# Create new column names, preserving 'Wavelength' and formatting others
new_columns_pyr = ['Wavelength']
for col in current_columns_pyr[1:]:
    if isinstance(col, (float, int)):
        new_columns_pyr.append(f'PYR_{col:.4f}mM_Absorbance') # Format to 4 decimal places for consistency
    else:
        new_columns_pyr.append(col)

wide_pyr_df.columns = new_columns_pyr

# Display the head of the wide DataFrame
print("Wide Pyruvate DataFrame Head:")
display(wide_pyr_df.head())

# 4. Save the wide_pyr_df DataFrame to a CSV file named 'Pyruvate_Concentration_Absorbance.csv'.
#    Ensure the index is not written to the CSV file.
output_filename_pyr = 'Pyruvate_Concentration_Absorbance.csv'
wide_pyr_df.to_csv(output_filename_pyr, index=False)
print(f"\nWide Pyruvate DataFrame saved to {output_filename_pyr}")

Wide Pyruvate DataFrame Head:


,Wavelength,PYR_1.0000mM_Absorbance,PYR_10.0000mM_Absorbance,PYR_50.0000mM_Absorbance,PYR_100.0000mM_Absorbance
0,190.0,1.978241,2.898288,3.304986,3.593821
1,191.0,1.979834,2.919439,3.638203,3.240218
2,192.0,1.973983,3.276395,3.784174,3.926600
3,193.0,1.962775,3.308305,3.880680,3.991495
4,194.0,1.933148,3.318090,3.712040,3.964487



Wide Pyruvate DataFrame saved to Pyruvate_Concentration_Absorbance.csv


---
<a id='extinction'></a>
## Build Extinction Coefficient Standards

Create the standards matrix (`standards_df`) containing extinction coefficients for NADH and pyruvate at each wavelength. This matrix is used for spectral deconvolution.

## First validate linear ranges for NADH and for pyruvate

In [11]:
import pandas as pd

# Filter merged_df for NADH data, Absorbance <= 2.5, and Expected_mM is not NaN
nadh_filtered_df = merged_df[
    (merged_df['Compound'] == 'NADH') &
    (merged_df['Absorbance'] <= 2.5) &
    (merged_df['Expected_mM'].notna())
].copy()

# Display the first 5 rows and the shape of the filtered DataFrame
print("NADH Filtered DataFrame Head:")
display(nadh_filtered_df.head())
print(f"\nShape of NADH Filtered DataFrame: {nadh_filtered_df.shape}")

NADH Filtered DataFrame Head:


,Wavelength,Absorbance,Compound,File_Name,Expected_mM,Concentration_mM
22,212.0,2.465739,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601
23,213.0,2.380679,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601
24,214.0,2.257281,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601
25,215.0,2.064577,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601
26,216.0,1.829588,NADH,0_05MM NADH SPECTRUM.WAV,0.05,0.044601



Shape of NADH Filtered DataFrame: (3390, 6)


## Analyze NADH Data

In [12]:
# Measure NADH concentrations based on a subset of wavelengths.
# Calculate the extinction coefficient, and R^2 value
# This will help us decide which wavelengths to use for measuring NADH concentrations
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

# Initialize a list to store the regression results
results_list = []

# Get the unique wavelengths present in the filtered data
unique_wavelengths = nadh_filtered_df['Wavelength'].unique()

# Iterate through each wavelength
for wav in unique_wavelengths:
    # Subset the data for the current wavelength
    subset = nadh_filtered_df[nadh_filtered_df['Wavelength'] == wav]

    # Require at least 2 data points to perform regression
    if len(subset) >= 2:
        try:
            # Fit OLS model: Absorbance ~ Expected_mM (no intercept)
            model = smf.ols('Absorbance ~ 0 + Expected_mM', data=subset).fit()

            # Extract slope and R-squared
            slope = model.params['Expected_mM']
            r_squared = model.rsquared

            # Append to results
            results_list.append({
                'Wavelength': wav,
                'Slope': slope,
                'R_squared': r_squared,
                'Num_Points': len(subset)
            })
        except Exception as e:
            print(f"Error fitting model at {wav} nm: {e}")

# Create a DataFrame from the results
nadh_linearity_df = pd.DataFrame(results_list)

# Sort by Wavelength
nadh_linearity_df = nadh_linearity_df.sort_values('Wavelength').reset_index(drop=True)

# Display the first few rows of the results
print("NADH Linearity Analysis Results (Head):")
display(nadh_linearity_df.head())

NADH Linearity Analysis Results (Head):


,Wavelength,Slope,R_squared,Num_Points
0,219.0,16.907245,0.985095,2
1,220.0,14.991107,0.989265,2
2,221.0,13.151934,0.991832,2
3,222.0,11.568007,0.993651,2
4,223.0,9.951321,0.997973,3


In [13]:
# Adjust the extinction coefficients assuming that the correct value at 340 nm is 6.22
import numpy as np

# 1. Find the slope at 340 nm
# Using boolean indexing to find the row where Wavelength is 340
slope_340_row = nadh_linearity_df[nadh_linearity_df['Wavelength'] == 340]

if not slope_340_row.empty:
    slope_340 = slope_340_row['Slope'].values[0]

    # 2. Define the target extinction coefficient
    target_extinction_340 = 6.22

    # 3. Calculate the scaling factor
    # Scaling Factor = Target / Actual Slope
    scaling_factor = target_extinction_340 / slope_340

    # 4. Add the 'Extinction_Coefficient' column to the DataFrame
    nadh_linearity_df['Extinction_Coefficient'] = nadh_linearity_df['Slope'] * scaling_factor

    print(f"Slope at 340 nm: {slope_340:.4f}")
    print(f"Scaling Factor (Target 6.22 / Slope 340nm): {scaling_factor:.4f}")
    print("\nFirst 5 rows with Extinction Coefficient:")
    display(nadh_linearity_df.head())
else:
    print("Error: Data for 340 nm not found in the linearity DataFrame.")

Slope at 340 nm: 5.5483
Scaling Factor (Target 6.22 / Slope 340nm): 1.1211

First 5 rows with Extinction Coefficient:


,Wavelength,Slope,R_squared,Num_Points,Extinction_Coefficient
0,219.0,16.907245,0.985095,2,18.954002
1,220.0,14.991107,0.989265,2,16.805900
2,221.0,13.151934,0.991832,2,14.744080
3,222.0,11.568007,0.993651,2,12.968407
4,223.0,9.951321,0.997973,3,11.156008


In [14]:
# Plot the normalized extinction coefficients
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if 'Extinction_Coefficient' in nadh_linearity_df.columns:
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=("Extinction Coefficient vs. Wavelength", "R-squared vs. Wavelength")
    )

    # Trace 1: Extinction Coefficient vs. Wavelength
    fig.add_trace(
        go.Scatter(
            x=nadh_linearity_df['Wavelength'],
            y=nadh_linearity_df['Extinction_Coefficient'],
            mode='lines',
            name='Extinction Coefficient',
            line=dict(color='purple')
        ),
        row=1, col=1
    )

    # Trace 2: R-squared vs. Wavelength
    fig.add_trace(
        go.Scatter(
            x=nadh_linearity_df['Wavelength'],
            y=nadh_linearity_df['R_squared'],
            mode='lines',
            name='R-squared',
            line=dict(color='green')
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        height=800,
        title_text="NADH Analysis: Extinction Coefficient (Normalized to 6.22 at 340 nm) and R-squared",
        showlegend=False
    )

    # Update axis titles
    fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=1)
    fig.update_yaxes(title_text="Extinction Coefficient (mM⁻¹ cm⁻¹)", row=1, col=1)
    fig.update_yaxes(title_text="R-squared", row=2, col=1)

    fig.show()
else:
    print("Extinction Coefficient column missing. Please check the calculation step.")

Perform a linearity analysis for Pyruvate (PYR) across all wavelengths. First, filter the `merged_df` to include only Pyruvate data with 'Absorbance' $\le$ 2.5 and valid 'Expected_mM'. Then, iterate through each unique wavelength to perform a linear regression (forcing the intercept to zero) of 'Absorbance' vs. 'Expected_mM'. Calculate the slope and R-squared for each wavelength. Finally, generate a plot with two subplots: one showing the Slope vs. Wavelength and the other showing R-squared vs. Wavelength to assess sensitivity and linearity.

## Analyze Pyruvate Data
Filter the merged DataFrame to include only valid Pyruvate data suitable for linearity analysis.


In [15]:
# Filter merged_df for Pyruvate data, Absorbance <= 2, and Expected_mM is not NaN
pyr_filtered_df = merged_df[
    (merged_df['Compound'] == 'PYR') &
    (merged_df['Absorbance'] <= 2) &
    (merged_df['Expected_mM'].notna())
].copy()

# Display the first 5 rows and the shape of the filtered DataFrame
print("Pyruvate Filtered DataFrame Head:")
display(pyr_filtered_df.head())
print(f"\nShape of Pyruvate Filtered DataFrame: {pyr_filtered_df.shape}")

Pyruvate Filtered DataFrame Head:


,Wavelength,Absorbance,Compound,File_Name,Expected_mM,Concentration_mM
3644,190.0,1.978241,PYR,1MM PYR SPECTRUM.WAV,1.0,1.0
3645,191.0,1.979834,PYR,1MM PYR SPECTRUM.WAV,1.0,1.0
3646,192.0,1.973983,PYR,1MM PYR SPECTRUM.WAV,1.0,1.0
3647,193.0,1.962775,PYR,1MM PYR SPECTRUM.WAV,1.0,1.0
3648,194.0,1.933148,PYR,1MM PYR SPECTRUM.WAV,1.0,1.0



Shape of Pyruvate Filtered DataFrame: (3415, 6)


In [16]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

# Initialize a list to store the regression results
pyr_results_list = []

# Get the unique wavelengths present in the filtered data
unique_wavelengths_pyr = pyr_filtered_df['Wavelength'].unique()

# Iterate through each wavelength
for wav in unique_wavelengths_pyr:
    # Subset the data for the current wavelength
    subset = pyr_filtered_df[pyr_filtered_df['Wavelength'] == wav]

    # Require at least 2 data points to perform regression
    if len(subset) >= 2:
        try:
            # Fit OLS model: Absorbance ~ Expected_mM (no intercept)
            model = smf.ols('Absorbance ~ 0 + Expected_mM', data=subset).fit()

            # Extract slope and R-squared
            slope = model.params['Expected_mM']
            r_squared = model.rsquared

            # Append to results
            pyr_results_list.append({
                'Wavelength': wav,
                'Slope': slope,
                'R_squared': r_squared,
                'Num_Points': len(subset)
            })
        except Exception as e:
            print(f"Error fitting model at {wav} nm: {e}")

# Create a DataFrame from the results
pyr_linearity_df = pd.DataFrame(pyr_results_list)

# Sort by Wavelength
pyr_linearity_df = pyr_linearity_df.sort_values('Wavelength').reset_index(drop=True)

# Display the first few rows of the results
print("Pyruvate Linearity Analysis Results (Head):")
display(pyr_linearity_df.head())

Pyruvate Linearity Analysis Results (Head):


,Wavelength,Slope,R_squared,Num_Points
0,243.0,0.188171,0.999991,2
1,244.0,0.172990,0.999997,2
2,245.0,0.158551,0.999999,2
3,246.0,0.145141,1.000000,2
4,247.0,0.132951,0.999999,2


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Pyruvate Slope (Extinction Coefficient) vs. Wavelength", "Pyruvate R-squared vs. Wavelength")
)

# Trace 1: Slope vs. Wavelength
fig.add_trace(
    go.Scatter(
        x=pyr_linearity_df['Wavelength'],
        y=pyr_linearity_df['Slope'],
        mode='lines',
        name='Slope (Extinction Coeff.)',
        line=dict(color='orange')
    ),
    row=1, col=1
)

# Trace 2: R-squared vs. Wavelength
fig.add_trace(
    go.Scatter(
        x=pyr_linearity_df['Wavelength'],
        y=pyr_linearity_df['R_squared'],
        mode='lines',
        name='R-squared',
        line=dict(color='blue')
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Pyruvate Linearity Analysis: Slope and R-squared vs. Wavelength",
    showlegend=False
)

# Update axis titles
fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=1)
fig.update_yaxes(title_text="Slope (mM⁻¹)", row=1, col=1)
fig.update_yaxes(title_text="R-squared", row=2, col=1)

fig.show()

Create a unified `standards_df` by merging `nadh_linearity_df` (using 'Extinction_Coefficient' as the NADH coefficient) and `pyr_linearity_df` (using 'Slope' as the Pyruvate coefficient) on 'Wavelength'. Then, define a function `calculate_concentrations(input_df)` that takes a dataframe containing 'Wavelength' and 'Absorbance', aligns it with the standards, and performs a linear regression ($Abs = [NADH]\cdot\epsilon_{NADH} + [PYR]\cdot\epsilon_{PYR}$) to solve for the concentrations. Finally, validate the model by testing it on one known NADH sample and one known Pyruvate sample from `merged_df`, displaying the calculated versus expected concentrations and the $R^2$ fit metric.

# Optimize data processing parameters

## Create Standards Matrix
Merge NADH and Pyruvate linearity data into a single standards dataframe containing extinction coefficients for both compounds.


In [18]:
import pandas as pd

# 1. Create nadh_standards
# Select 'Wavelength' and 'Extinction_Coefficient' and rename
if 'Extinction_Coefficient' in nadh_linearity_df.columns:
    nadh_standards = nadh_linearity_df[['Wavelength', 'Extinction_Coefficient']].copy()
    nadh_standards = nadh_standards.rename(columns={'Extinction_Coefficient': 'NADH_Coeff'})
else:
    print("Error: 'Extinction_Coefficient' column missing in nadh_linearity_df")
    nadh_standards = pd.DataFrame() # Create empty to avoid crash in merge if error occurs

# 2. Create pyr_standards
# Select 'Wavelength' and 'Slope' and rename
pyr_standards = pyr_linearity_df[['Wavelength', 'Slope']].copy()
pyr_standards = pyr_standards.rename(columns={'Slope': 'PYR_Coeff'})

# 3. Merge nadh_standards and pyr_standards
# Use inner join on 'Wavelength'
standards_df = pd.merge(nadh_standards, pyr_standards, on='Wavelength', how='inner')

# 4. Display results
print("Standards DataFrame Head:")
display(standards_df.head())
print(f"\nShape of Standards DataFrame: {standards_df.shape}")

Standards DataFrame Head:


,Wavelength,NADH_Coeff,PYR_Coeff
0,243.0,8.371430,0.188171
1,244.0,8.736970,0.172990
2,245.0,9.114069,0.158551
3,246.0,9.543467,0.145141
4,247.0,10.003543,0.132951



Shape of Standards DataFrame: (858, 3)


In [19]:
output_filename_standards = 'NADH_Pyruvate_Standards.csv'
standards_df.to_csv(output_filename_standards, index=False)
print(f"Standards DataFrame saved to {output_filename_standards}")

Standards DataFrame saved to NADH_Pyruvate_Standards.csv


---
<a id='kinetic'></a>
## Load Kinetic Data

Load time course data from kinetic experiments.

In [20]:
import pandas as pd

# Define the full file path provided in the instructions
file_path = '/content/drive/MyDrive/Research/PDC+ADH+FDH assay data Evelyn 2025/KD files from Agilent spec/1229 PDC PYRUVATE 100MM-8.csv'

try:
    # Load the kinetic data file
    kinetic_df = pd.read_csv(file_path)

    # Display the first 10 rows to inspect for metadata
    print("Kinetic Data Head (first 10 rows):")
    display(kinetic_df.head(10))
    print(f"\nShape of Kinetic DataFrame: {kinetic_df.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the path.")

Kinetic Data Head (first 10 rows):


,sample,Time_s,190,191,192,193,194,195,196,197,...,1091,1092,1093,1094,1095,1096,1097,1098,1099,filename
0,CELL_1,1.4,-0.040190,-0.025164,-0.020460,0.002255,0.007010,-0.042197,-0.013320,0.015717,...,-0.015170,-0.015754,-0.014958,-0.017492,-0.016090,-0.014441,-0.018435,-0.019453,-0.019712,1229 PDC PYRUVATE 100MM-8.KD
1,CELL_1,7.0,-0.035162,-0.015901,-0.045796,-0.000527,0.017020,-0.041877,-0.011095,-0.005792,...,-0.015467,-0.017028,-0.018837,-0.018133,-0.019844,-0.014425,-0.020117,-0.017702,-0.018785,1229 PDC PYRUVATE 100MM-8.KD
2,CELL_1,13.5,-0.025568,-0.025858,-0.040857,-0.017770,-0.007006,-0.029105,0.000960,-0.017584,...,-0.015920,-0.016428,-0.015409,-0.017478,-0.018126,-0.013627,-0.019478,-0.017424,-0.018097,1229 PDC PYRUVATE 100MM-8.KD
3,CELL_1,19.9,-0.035588,-0.045554,-0.058393,0.002479,-0.019413,-0.049197,-0.021289,-0.010043,...,-0.014615,-0.017081,-0.017593,-0.018870,-0.019313,-0.012831,-0.020371,-0.019111,-0.019593,1229 PDC PYRUVATE 100MM-8.KD
4,CELL_1,26.2,-0.030612,-0.030499,-0.030189,0.002214,0.002298,-0.031126,-0.013102,-0.010210,...,-0.015845,-0.016278,-0.019495,-0.016925,-0.017804,-0.012013,-0.020674,-0.018943,-0.016878,1229 PDC PYRUVATE 100MM-8.KD
5,CELL_1,32.7,-0.035280,-0.025367,-0.020052,-0.000019,-0.022268,-0.035635,-0.027234,-0.001531,...,-0.015112,-0.020629,-0.017226,-0.017594,-0.016663,-0.012801,-0.019484,-0.017867,-0.018404,1229 PDC PYRUVATE 100MM-8.KD
6,CELL_1,39.0,-0.030412,-0.025740,-0.029966,-0.000128,-0.025808,-0.014565,-0.014451,0.003623,...,-0.013878,-0.017083,-0.016978,-0.016145,-0.017808,-0.012908,-0.019199,-0.016729,-0.015973,1229 PDC PYRUVATE 100MM-8.KD
7,CELL_1,45.5,-0.005201,-0.031645,-0.049096,0.014328,0.011323,-0.037076,-0.008773,0.014840,...,-0.013252,-0.017771,-0.015123,-0.017083,-0.017722,-0.012039,-0.018786,-0.016330,-0.018835,1229 PDC PYRUVATE 100MM-8.KD
8,CELL_1,52.1,-0.040246,-0.030464,-0.040719,-0.005971,0.017302,-0.041159,-0.000295,0.004899,...,-0.015264,-0.014437,-0.015357,-0.014946,-0.015792,-0.012037,-0.018260,-0.016474,-0.018178,1229 PDC PYRUVATE 100MM-8.KD
9,CELL_1,58.2,-0.025513,-0.015899,-0.035326,0.007373,-0.002850,-0.025620,-0.014807,-0.013025,...,-0.014754,-0.014239,-0.018074,-0.016269,-0.016409,-0.012829,-0.017312,-0.017653,-0.018355,1229 PDC PYRUVATE 100MM-8.KD



Shape of Kinetic DataFrame: (333, 913)


---
<a id='example'></a>
## Process CELL_1 Data

Demonstrate the complete workflow by processing time course data from CELL_1.

In [21]:
import plotly.express as px
import pandas as pd

# Filter for CELL_1 samples
cell_1_df = kinetic_df[kinetic_df['sample'] == 'CELL_1'].copy()

# 1. Plot Abs340 Kinetics
# Ensure the column name is treated correctly (string '340')
col_340 = '340'

if col_340 in cell_1_df.columns:
    fig_abs340 = px.line(
        cell_1_df,
        x='Time_s',
        y=col_340,
        title='Absorbance at 340 nm vs. Time (CELL_1)',
        markers=True
    )
    fig_abs340.update_layout(xaxis_title='Time (s)', yaxis_title='Absorbance (340 nm)')
    fig_abs340.show()

    # 2. Calculate NADH (Single-Wavelength)
    blank = 1.68
    extinction_coeff_340 = 6.22

    # Calculate concentration
    cell_1_df['NADH_Conc_SingleWav'] = (cell_1_df[col_340] - blank) / extinction_coeff_340

    print("NADH Concentration (Single-Wavelength) calculated.")
    display(cell_1_df[['Time_s', col_340, 'NADH_Conc_SingleWav']].head())
else:
    print(f"Column '{col_340}' not found in DataFrame. Available columns: {cell_1_df.columns[:10]}...")

NADH Concentration (Single-Wavelength) calculated.


,Time_s,340,NADH_Conc_SingleWav
0,1.4,-0.003061,-0.270589
1,7.0,-0.002829,-0.270551
2,13.5,-0.002923,-0.270566
3,19.9,-0.002592,-0.270513
4,26.2,-0.002926,-0.270567


---
<a id='functions'></a>
## Core Analysis Functions

### Spectral Deconvolution Function

The `calculate_concentrations()` function performs spectral deconvolution to determine NADH and/or pyruvate concentrations from absorbance spectra.

**Key Features:**
- Wavelength range filtering
- Absorbance threshold filtering
- Optional fixed concentration mode (fix NADH or pyruvate, solve for the other)
- Built-in visualization
- Baseline correction via intercept fitting

In [22]:
# =============================================================================
# ENHANCED calculate_concentrations FUNCTION WITH PLOTLY VISUALIZATION
# AND FIXED CONCENTRATION FEATURE
# =============================================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def calculate_concentrations(
    spectrum_df,
    standards_df,
    wavelength_range=(320, 420),
    absorbance_max=2,
    fit_intercept=False,
    fixed_nadh=None,
    fixed_pyr=None,
    plot=False,
    plot_title=None
):
    """
    Calculates concentrations of NADH and Pyruvate using linear regression against standards.
    Optionally fixes one component concentration and solves for the other.
    Optionally generates an interactive Plotly visualization of the spectral decomposition.

    Args:
        spectrum_df: DataFrame containing 'Wavelength' and 'Absorbance'.
        standards_df: DataFrame containing 'Wavelength', 'NADH_Coeff', and 'PYR_Coeff'.
        wavelength_range: Tuple (min_nm, max_nm) to filter data. Default is None.
        absorbance_max: Float value to exclude absorbance readings above this limit. Default is None.
        fit_intercept: Boolean. If True, allows a non-zero intercept (baseline offset). Default is False.
        fixed_nadh: Float. If provided, fixes NADH concentration (mM) and solves for Pyruvate only.
        fixed_pyr: Float. If provided, fixes Pyruvate concentration (mM) and solves for NADH only.
        plot: Boolean. If True, generates and displays a Plotly figure. Default is False.
        plot_title: String. Custom title for the plot. If None, auto-generates title.

    Returns:
        dict: {
            'NADH_Conc': float,           # Calculated/fixed NADH concentration (mM)
            'PYR_Conc': float,            # Calculated/fixed Pyruvate concentration (mM)
            'Intercept': float,           # Baseline offset (if fit_intercept=True)
            'R_squared': float,           # Goodness of fit
            'fixed_component': str or None, # Which component was fixed ('NADH', 'PYR', or None)
            'wavelengths': np.array,      # Wavelength values used in fit
            'raw_absorbance': np.array,   # Raw absorbance data
            'fitted_absorbance': np.array, # Total fitted absorbance
            'nadh_component': np.array,   # NADH contribution to total absorbance
            'pyr_component': np.array,    # Pyruvate contribution to total absorbance
            'intercept_component': np.array, # Baseline contribution (if applicable)
            'residuals': np.array,        # Fit residuals
            'fig': plotly.graph_objects.Figure or None  # Plotly figure (if plot=True)
        }

    Notes:
        - Cannot specify both fixed_nadh and fixed_pyr simultaneously
        - When a component is fixed, R² reflects the quality of fit for the remaining component(s)
    """

    # Validate that both concentrations are not fixed simultaneously
    if fixed_nadh is not None and fixed_pyr is not None:
        raise ValueError("Cannot fix both NADH and Pyruvate concentrations simultaneously. "
                        "Specify only one or neither.")

    # Merge spectrum with standards on Wavelength
    merged_data = pd.merge(spectrum_df, standards_df, on='Wavelength', how='inner')

    # Filter by Wavelength Range if provided
    if wavelength_range:
        min_wav, max_wav = wavelength_range
        merged_data = merged_data[
            (merged_data['Wavelength'] >= min_wav) &
            (merged_data['Wavelength'] <= max_wav)
        ]

    # Filter by Absorbance Max if provided
    if absorbance_max is not None:
        merged_data = merged_data[merged_data['Absorbance'] <= absorbance_max]

    if merged_data.empty or len(merged_data) < 2:
        return {
            'NADH_Conc': None,
            'PYR_Conc': None,
            'Intercept': 0,
            'R_squared': None,
            'fixed_component': None,
            'wavelengths': None,
            'raw_absorbance': None,
            'fitted_absorbance': None,
            'nadh_component': None,
            'pyr_component': None,
            'intercept_component': None,
            'residuals': None,
            'fig': None
        }

    wavelengths = merged_data['Wavelength'].values
    y_original = merged_data['Absorbance'].values

    # Determine fitting mode
    fixed_component = None

    if fixed_nadh is not None:
        # NADH is fixed, solve for Pyruvate (and optionally intercept)
        fixed_component = 'NADH'
        nadh_conc = fixed_nadh

        # Subtract NADH contribution from observed absorbance
        nadh_contribution = fixed_nadh * merged_data['NADH_Coeff'].values
        y_adjusted = y_original - nadh_contribution

        # Fit only Pyruvate (and optionally intercept)
        X = merged_data[['PYR_Coeff']].copy()
        if fit_intercept:
            X = sm.add_constant(X)

        model = sm.OLS(y_adjusted, X).fit()
        pyr_conc = model.params.get('PYR_Coeff', 0)
        intercept = model.params.get('const', 0)
        r_squared = model.rsquared

    elif fixed_pyr is not None:
        # Pyruvate is fixed, solve for NADH (and optionally intercept)
        fixed_component = 'PYR'
        pyr_conc = fixed_pyr

        # Subtract Pyruvate contribution from observed absorbance
        pyr_contribution = fixed_pyr * merged_data['PYR_Coeff'].values
        y_adjusted = y_original - pyr_contribution

        # Fit only NADH (and optionally intercept)
        X = merged_data[['NADH_Coeff']].copy()
        if fit_intercept:
            X = sm.add_constant(X)

        model = sm.OLS(y_adjusted, X).fit()
        nadh_conc = model.params.get('NADH_Coeff', 0)
        intercept = model.params.get('const', 0)
        r_squared = model.rsquared

    else:
        # Standard mode: fit both NADH and Pyruvate
        fixed_component = None
        X = merged_data[['NADH_Coeff', 'PYR_Coeff']].copy()

        if fit_intercept:
            X = sm.add_constant(X)

        model = sm.OLS(y_original, X).fit()
        nadh_conc = model.params.get('NADH_Coeff', 0)
        pyr_conc = model.params.get('PYR_Coeff', 0)
        intercept = model.params.get('const', 0)
        r_squared = model.rsquared

    # Calculate component contributions
    nadh_component = nadh_conc * merged_data['NADH_Coeff'].values
    pyr_component = pyr_conc * merged_data['PYR_Coeff'].values
    intercept_component = np.full_like(wavelengths, intercept) if fit_intercept else np.zeros_like(wavelengths)

    # Total fitted absorbance
    fitted_absorbance = nadh_component + pyr_component + intercept_component

    # Residuals
    residuals = y_original - fitted_absorbance

    # Prepare return dictionary
    result = {
        'NADH_Conc': nadh_conc,
        'PYR_Conc': pyr_conc,
        'Intercept': intercept,
        'R_squared': r_squared,
        'fixed_component': fixed_component,
        'wavelengths': wavelengths,
        'raw_absorbance': y_original,
        'fitted_absorbance': fitted_absorbance,
        'nadh_component': nadh_component,
        'pyr_component': pyr_component,
        'intercept_component': intercept_component,
        'residuals': residuals,
        'fig': None
    }

    # Generate plot if requested
    if plot:
        fig = _create_deconvolution_plot(result, wavelength_range, plot_title)
        result['fig'] = fig
        fig.show()

    return result


def _create_deconvolution_plot(result, wavelength_range, plot_title):
    """
    Internal function to create a comprehensive Plotly visualization of spectral deconvolution.

    Args:
        result: Dictionary output from calculate_concentrations
        wavelength_range: Tuple of (min, max) wavelengths or None
        plot_title: Custom plot title or None

    Returns:
        plotly.graph_objects.Figure
    """
    wavelengths = result['wavelengths']
    fixed_component = result['fixed_component']

    # Create subplots: main spectrum plot + residuals plot
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.75, 0.25],
        subplot_titles=('Spectral Decomposition', 'Fit Residuals'),
        vertical_spacing=0.12,
        specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
    )

    # --- Main Plot: Spectral Components ---

    # Raw data
    fig.add_trace(
        go.Scatter(
            x=wavelengths,
            y=result['raw_absorbance'],
            mode='markers',
            name='Raw Data',
            marker=dict(size=4, color='rgba(100, 100, 100, 0.6)', symbol='circle'),
            legendgroup='data',
            showlegend=True
        ),
        row=1, col=1
    )

    # Total fit
    fig.add_trace(
        go.Scatter(
            x=wavelengths,
            y=result['fitted_absorbance'],
            mode='lines',
            name=f'Total Fit (R² = {result["R_squared"]:.6f})',
            line=dict(color='black', width=2.5),
            legendgroup='fit',
            showlegend=True
        ),
        row=1, col=1
    )

    # NADH component (with indicator if fixed)
    nadh_label = f'NADH Component ({result["NADH_Conc"]:.4f} mM)'
    if fixed_component == 'NADH':
        nadh_label += ' [FIXED]'

    fig.add_trace(
        go.Scatter(
            x=wavelengths,
            y=result['nadh_component'],
            mode='lines',
            name=nadh_label,
            line=dict(
                color='royalblue',
                width=3 if fixed_component == 'NADH' else 2,
                dash='solid' if fixed_component == 'NADH' else 'dash'
            ),
            legendgroup='components',
            showlegend=True
        ),
        row=1, col=1
    )

    # Pyruvate component (with indicator if fixed)
    pyr_label = f'Pyruvate Component ({result["PYR_Conc"]:.4f} mM)'
    if fixed_component == 'PYR':
        pyr_label += ' [FIXED]'

    fig.add_trace(
        go.Scatter(
            x=wavelengths,
            y=result['pyr_component'],
            mode='lines',
            name=pyr_label,
            line=dict(
                color='crimson',
                width=3 if fixed_component == 'PYR' else 2,
                dash='solid' if fixed_component == 'PYR' else 'dash'
            ),
            legendgroup='components',
            showlegend=True
        ),
        row=1, col=1
    )

    # Intercept/Baseline component (if non-zero)
    if result['Intercept'] != 0:
        fig.add_trace(
            go.Scatter(
                x=wavelengths,
                y=result['intercept_component'],
                mode='lines',
                name=f'Baseline ({result["Intercept"]:.4f})',
                line=dict(color='gray', width=1.5, dash='dot'),
                legendgroup='components',
                showlegend=True
            ),
            row=1, col=1
        )

    # --- Residuals Plot ---
    fig.add_trace(
        go.Scatter(
            x=wavelengths,
            y=result['residuals'],
            mode='markers',
            name='Residuals',
            marker=dict(size=3, color='orange', symbol='circle'),
            showlegend=False
        ),
        row=2, col=1
    )

    # Zero line for residuals
    fig.add_trace(
        go.Scatter(
            x=[wavelengths.min(), wavelengths.max()],
            y=[0, 0],
            mode='lines',
            line=dict(color='black', width=1, dash='dash'),
            showlegend=False
        ),
        row=2, col=1
    )

    # --- Layout Configuration ---

    # Auto-generate title if not provided
    if plot_title is None:
        wavelength_str = f" ({wavelength_range[0]}-{wavelength_range[1]} nm)" if wavelength_range else ""
        fixed_str = ""
        if fixed_component:
            fixed_value = result['NADH_Conc'] if fixed_component == 'NADH' else result['PYR_Conc']
            fixed_str = f" [{fixed_component} Fixed at {fixed_value:.4f} mM]"
        plot_title = f'Spectral Deconvolution{wavelength_str}{fixed_str}'

    fig.update_layout(
        title=dict(
            text=plot_title,
            font=dict(size=16, color='black'),
            x=0.5,
            xanchor='center'
        ),
        height=700,
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.98,
            xanchor="right",
            x=0.98,
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="gray",
            borderwidth=1
        ),
        font=dict(size=12)
    )

    # X-axis labels
    fig.update_xaxes(title_text="Wavelength (nm)", row=1, col=1, showgrid=True)
    fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=1, showgrid=True)

    # Y-axis labels
    fig.update_yaxes(title_text="Absorbance", row=1, col=1, showgrid=True)
    fig.update_yaxes(title_text="Residual", row=2, col=1, showgrid=True, zeroline=True)

    return fig

In [23]:
import pandas as pd
import numpy as np

def extract_spectrum_at_time(df, target_time, min_wavelength, max_wavelength):
    """
    Finds the row closest to target_time in df and extracts the spectrum
    within the specified wavelength range.
    """
    # Find closest row
    if 'Time_s' not in df.columns:
        raise ValueError("DataFrame must contain 'Time_s' column")

    closest_idx = (df['Time_s'] - target_time).abs().idxmin()
    closest_row = df.loc[closest_idx]
    actual_time = closest_row['Time_s']

    print(f"Target Time: {target_time} s")
    print(f"Actual Time Found: {actual_time} s")

    # Identify spectral columns
    # Heuristic: columns that can be converted to float are wavelengths
    # Exclude known metadata just in case
    metadata_cols = ['sample', 'Time_s', 'filename', 'NADH_Conc_SingleWav', 'NADH_Conc_Spectral']
    potential_cols = [c for c in df.columns if c not in metadata_cols]

    selected_wavelengths = []
    selected_cols = []

    for col in potential_cols:
        try:
            wav = float(col)
            if min_wavelength <= wav <= max_wavelength:
                selected_wavelengths.append(wav)
                selected_cols.append(col)
        except ValueError:
            continue

    # Extract values
    absorbances = closest_row[selected_cols].values

    # Create DataFrame
    spectrum_df = pd.DataFrame({
        'Wavelength': selected_wavelengths,
        'Absorbance': absorbances
    })

    spectrum_df['Absorbance'] = pd.to_numeric(spectrum_df['Absorbance'], errors='coerce')

    return spectrum_df

# Usage
spectrum_602_df = extract_spectrum_at_time(cell_1_df, 602.2, 190, 1100)

# Display the DataFrame
print("\nSpectrum at ~602.2s (190-1100 nm):")
display(spectrum_602_df.head())
print(f"Shape: {spectrum_602_df.shape}")

Target Time: 602.2 s
Actual Time Found: 602.2 s

Spectrum at ~602.2s (190-1100 nm):


,Wavelength,Absorbance
0,190.0,0.000395
1,191.0,0.027115
2,192.0,0.016452
3,193.0,0.032912
4,194.0,0.025004


Shape: (910, 2)


## Calculate Components and Plot Spectral Decomposition
Calculate concentrations and theoretical curves for the extracted spectrum, then visualize the decomposition compared to raw data.


In [24]:
wavelength_min = 320
wavelength_max = 420
absorbance_max = 2

result_180s = calculate_concentrations(
    spectrum_df=extract_spectrum_at_time(cell_1_df, 180, wavelength_min, wavelength_max),
    standards_df=standards_df,
    wavelength_range=(wavelength_min, wavelength_max),
    absorbance_max=absorbance_max,
    fit_intercept=False,
    plot=True,  # Enable plotting!
    plot_title="Spectral Deconvolution at 180s",
    #fixed_pyr = 100
)

Target Time: 180 s
Actual Time Found: 179.9 s


In [25]:
wavelength_min = 360
wavelength_max = 420
absorbance_max = 2
time = 333

result_180s = calculate_concentrations(
    spectrum_df=extract_spectrum_at_time(cell_1_df, time, wavelength_min, wavelength_max),
    standards_df=standards_df,
    wavelength_range=(wavelength_min, wavelength_max),
    absorbance_max=absorbance_max,
    fit_intercept=False,
    plot=True,  # Enable plotting!
    plot_title=f"Spectral Deconvolution at {time}s",
    fixed_pyr = 99.11
)

Target Time: 333 s
Actual Time Found: 333.4 s


# Process one example of PDC timecourse data

---
<a id='timecourse'></a>
## PDC Time Course Analysis

### Automated Time Course Processing Function

The `process_pdc_timecourse()` function automates the analysis of PDC enzyme assay time courses.

**Key Features:**
- **Automatic pyruvate calculation** from blank spectrum (assuming NADH=0)
- **Optional parameters** - `blank_time` and `initial_pyruvate_mM` are both optional
- Multiple processing methods (single wavelength, full spectrum, constrained)
- Built-in diagnostic plots (NADH vs time, R², spectra)
- Initial rate calculation

**Convenience Wrapper:** `process_pdc_timecourse_simple()` - Just provide data and timing!

In [ ]:
# =============================================================================
# PDC TIME COURSE PROCESSING FUNCTION (FINAL VERSION)
# =============================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots



PDC time course processing functions loaded successfully!

Available functions:
  - process_pdc_timecourse()       : Full-featured function with all options
  - process_pdc_timecourse_simple(): Quick analysis (RECOMMENDED)

NEW: initial_pyruvate_mM is now OPTIONAL when blank_time is provided!
     Pyruvate is automatically calculated from blank spectrum.


In [ ]:
"""
# OLD VERSION
def process_pdc_timecourse(
    spectral_df,
    standards_df,
    assay_start_time,
    blank_time=None,
    initial_pyruvate_mM=None,
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    plot=False,
    plot_title=None,
    verbose=True
):
    """
    Process PDC (Pyruvate Dehydrogenase Complex) time course data to calculate NADH concentrations.

    This function processes spectrophotometric time course data from a PDC enzyme assay,
    calculating NADH concentrations at each time point using spectral deconvolution.

    **Pyruvate Concentration Logic:**
    - If blank_time is provided: Calculates pyruvate from blank spectrum (assuming NADH=0)
    - If blank_time is None: Must provide initial_pyruvate_mM manually
    - initial_pyruvate_mM is optional when blank_time is given (used only as fallback)

    Parameters
    ----------
    spectral_df : pandas.DataFrame
        DataFrame containing spectral time course data with:
        - 'Time_s' column: time in seconds
        - Wavelength columns: named with wavelength values (e.g., '340', '350', etc.)
        - Optional 'sample' column for identification

    standards_df : pandas.DataFrame
        DataFrame with extinction coefficient standards containing:
        - 'Wavelength' column: wavelength values
        - 'NADH_Coeff' column: NADH extinction coefficients (mM⁻¹cm⁻¹)
        - 'PYR_Coeff' column: Pyruvate extinction coefficients (mM⁻¹cm⁻¹)

    assay_start_time : float
        Time point (in seconds) when the assay was initiated (e.g., when substrate was added)

    blank_time : float, optional (default=None)
        Time point (in seconds) to use for blank/baseline correction
        If provided, pyruvate concentration is calculated from this spectrum (assuming NADH=0)
        If None, must provide initial_pyruvate_mM manually

    initial_pyruvate_mM : float, optional (default=None)
        Initial pyruvate concentration in mM
        - If blank_time is provided: Used only as fallback if calculation fails
        - If blank_time is None: REQUIRED - used directly for all calculations

    method : str, optional (default='constrained')
        Method for NADH calculation:
        - 'single_wavelength': Use Abs340 only with blank subtraction
        - 'full_spectrum': Use entire spectrum for deconvolution
        - 'constrained': Use wavelength_range and absorbance_max constraints (recommended)

    wavelength_range : tuple, optional (default=(320, 420))
        Tuple of (min_wavelength, max_wavelength) in nm
        Only used if method='constrained'

    absorbance_max : float, optional (default=2.5)
        Maximum absorbance threshold to exclude saturated data
        Only used if method='constrained'

    plot : bool, optional (default=False)
        If True, generates diagnostic plots

    plot_title : str, optional (default=None)
        Custom title for plots. Auto-generates if None.

    verbose : bool, optional (default=True)
        If True, prints progress and summary statistics

    Returns
    -------
    pandas.DataFrame
        DataFrame with the following columns:
        - 'Time_s': Time in seconds
        - 'Time_Relative_s': Time relative to assay_start_time (0 = assay start)
        - 'NADH_mM': NADH concentration in mM
        - 'Pyruvate_mM': Pyruvate concentration in mM (fixed value)
        - 'R_squared': Goodness of fit for each time point
        - 'Intercept': Baseline offset at each time point
        - 'Blank_Pyruvate_mM': Pyruvate concentration calculated from blank
        - 'Method': Method used for calculation
        - 'Fixed_Pyruvate': Whether pyruvate was fixed (boolean)

    Examples
    --------
    >>> # Recommended: Automatic pyruvate calculation from blank
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     blank_time=180.0,  # Pyruvate auto-calculated from this spectrum
    ...     plot=True
    ... )

    >>> # Manual pyruvate specification (no blank_time)
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     initial_pyruvate_mM=100.0,  # Must specify manually
    ...     plot=True
    ... )

    >>> # Provide fallback in case blank calculation fails
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     blank_time=180.0,
    ...     initial_pyruvate_mM=100.0,  # Used only if blank calc fails
    ...     plot=True
    ... )

    Notes
    -----
    - The function assumes a 1 cm path length
    - When blank_time is provided, pyruvate is calculated assuming NADH=0
    - For PDC assays, NADH production indicates pyruvate consumption
    - R² values < 0.95 may indicate poor fit quality
    """

    # Validate parameters
    if blank_time is None and initial_pyruvate_mM is None:
        raise ValueError(
            "Must provide either blank_time (for automatic pyruvate calculation) "
            "or initial_pyruvate_mM (for manual specification)"
        )

    if verbose:
        print("="*80)
        print("PDC TIME COURSE PROCESSING")
        print("="*80)
        print(f"Method: {method}")
        print(f"Assay start time: {assay_start_time} s")
        if blank_time is not None:
            print(f"Blank time: {blank_time} s (pyruvate will be calculated)")
            if initial_pyruvate_mM is not None:
                print(f"Fallback pyruvate: {initial_pyruvate_mM} mM (if calculation fails)")
        else:
            print(f"Manual pyruvate: {initial_pyruvate_mM} mM (no blank calculation)")
        if method == 'constrained':
            print(f"Wavelength range: {wavelength_range[0]}-{wavelength_range[1]} nm")
            print(f"Max absorbance: {absorbance_max}")
        print()

    # Identify wavelength columns
    non_spectral_cols = ['sample', 'Time_s', 'filename', 'NADH_Conc_SingleWav',
                         'NADH_Conc_Spectral', 'NADH_Method1', 'NADH_Method2',
                         'NADH_Method3', 'NADH_Method4']
    spectral_cols = [c for c in spectral_df.columns if c not in non_spectral_cols]

    # Determine pyruvate concentration to use
    if blank_time is not None and method != 'single_wavelength':
        # Calculate pyruvate from blank spectrum
        blank_pyruvate_mM = _calculate_blank_pyruvate(
            spectral_df, standards_df, spectral_cols, blank_time,
            method, wavelength_range, absorbance_max,
            initial_pyruvate_mM, verbose
        )
    elif initial_pyruvate_mM is not None:
        # Use manually specified pyruvate
        blank_pyruvate_mM = initial_pyruvate_mM
        if verbose and method != 'single_wavelength':
            print(f"Using manually specified pyruvate: {blank_pyruvate_mM:.4f} mM")
            print()
    else:
        raise ValueError("This should not happen - parameter validation failed")

    # Process based on method
    if method == 'single_wavelength':
        if blank_time is None:
            raise ValueError("single_wavelength method requires blank_time to be specified")
        results_df = _process_single_wavelength(
            spectral_df, blank_time, verbose
        )
        results_df['Blank_Pyruvate_mM'] = np.nan  # Not applicable
    else:
        results_df = _process_spectral_deconvolution(
            spectral_df, standards_df, spectral_cols,
            method, wavelength_range, absorbance_max,
            blank_pyruvate_mM,
            verbose
        )
        results_df['Blank_Pyruvate_mM'] = blank_pyruvate_mM

    # Add relative time column (0 = assay start)
    results_df['Time_Relative_s'] = results_df['Time_s'] - assay_start_time

    # Add metadata columns
    results_df['Method'] = method
    results_df['Fixed_Pyruvate'] = True if method != 'single_wavelength' else False
    results_df['Assay_Start_s'] = assay_start_time
    results_df['Blank_Time_s'] = blank_time if blank_time is not None else np.nan

    # Reorder columns for clarity
    col_order = ['Time_s', 'Time_Relative_s', 'NADH_mM', 'Pyruvate_mM',
                 'R_squared', 'Intercept', 'Blank_Pyruvate_mM',
                 'Method', 'Fixed_Pyruvate', 'Assay_Start_s', 'Blank_Time_s']
    results_df = results_df[[c for c in col_order if c in results_df.columns]]

    # Print summary statistics
    if verbose:
        _print_summary_statistics(results_df, assay_start_time)

    # Generate plots if requested
    if plot:
        _generate_diagnostic_plots(
            results_df, spectral_df, spectral_cols, standards_df,
            assay_start_time, plot_title, method
        )

    return results_df


def _calculate_blank_pyruvate(
    spectral_df, standards_df, spectral_cols, blank_time,
    method, wavelength_range, absorbance_max,
    fallback_pyruvate_mM, verbose
):
    """
    Calculate pyruvate concentration at blank_time assuming NADH=0.

    Returns the calculated pyruvate or fallback value if calculation fails.
    """
    if verbose:
        print("Calculating pyruvate concentration at blank time...")
        print(f"  Assumption: NADH = 0 mM at t = {blank_time} s")

    # Find the row closest to blank_time
    idx_blank = (spectral_df['Time_s'] - blank_time).abs().idxmin()
    blank_row = spectral_df.iloc[idx_blank]
    actual_blank_time = blank_row['Time_s']

    # Build spectrum dataframe for blank time point
    spectrum_data = []
    for col in spectral_cols:
        try:
            wavelength = float(col)
            absorbance = blank_row[col]
            spectrum_data.append({'Wavelength': wavelength, 'Absorbance': absorbance})
        except:
            continue

    spectrum_df_blank = pd.DataFrame(spectrum_data)

    # Set parameters based on method
    if method == 'full_spectrum':
        wl_range = None
        abs_max = None
    elif method == 'constrained':
        wl_range = wavelength_range
        abs_max = absorbance_max
    else:
        raise ValueError(f"Unknown method: {method}")

    # Calculate concentrations with NADH fixed at 0
    try:
        result = calculate_concentrations(
            spectrum_df=spectrum_df_blank,
            standards_df=standards_df,
            wavelength_range=wl_range,
            absorbance_max=abs_max,
            fit_intercept=True,
            fixed_nadh=0.0,  # Fix NADH at 0 for blank
            plot=False
        )

        blank_pyruvate = result['PYR_Conc']
        r_squared = result['R_squared']

        if verbose:
            print(f"  Actual blank time: {actual_blank_time:.1f} s")
            print(f"  Calculated pyruvate: {blank_pyruvate:.4f} mM")
            print(f"  Fit R²: {r_squared:.6f}")

            # Show difference from fallback if provided
            if fallback_pyruvate_mM is not None:
                diff = blank_pyruvate - fallback_pyruvate_mM
                pct_diff = 100 * diff / fallback_pyruvate_mM
                print(f"  Difference from fallback: {diff:+.4f} mM ({pct_diff:+.2f}%)")

            if r_squared < 0.95:
                print(f"  ⚠️  Warning: Low R² ({r_squared:.4f}) at blank time")

            print()

        return blank_pyruvate

    except Exception as e:
        if verbose:
            print(f"  ⚠️  Error calculating blank pyruvate: {e}")
            if fallback_pyruvate_mM is not None:
                print(f"  Using fallback value: {fallback_pyruvate_mM:.4f} mM")
            else:
                print(f"  ERROR: No fallback value provided!")
            print()

        if fallback_pyruvate_mM is not None:
            return fallback_pyruvate_mM
        else:
            raise ValueError(
                "Failed to calculate pyruvate from blank and no fallback value provided. "
                "Please provide initial_pyruvate_mM as a fallback."
            )


def _process_single_wavelength(spectral_df, blank_time, verbose):
    """Process using single wavelength (340 nm) method"""
    col_340 = '340'

    if col_340 not in spectral_df.columns:
        raise ValueError(f"Column '{col_340}' not found in spectral_df")

    # Get blank value
    idx_blank = (spectral_df['Time_s'] - blank_time).abs().idxmin()
    blank_340 = spectral_df.loc[idx_blank, col_340]

    # Calculate NADH
    epsilon_340 = 6.22  # mM^-1 cm^-1
    nadh_conc = (spectral_df[col_340] - blank_340) / epsilon_340

    if verbose:
        print(f"Single wavelength method:")
        print(f"  Blank Abs340: {blank_340:.6f}")
        print(f"  Extinction coeff: {epsilon_340} mM⁻¹cm⁻¹")
        print()

    # Create results dataframe
    results_df = pd.DataFrame({
        'Time_s': spectral_df['Time_s'],
        'NADH_mM': nadh_conc,
        'Pyruvate_mM': np.nan,
        'R_squared': np.nan,
        'Intercept': -blank_340
    })

    return results_df


def _process_spectral_deconvolution(
    spectral_df, standards_df, spectral_cols,
    method, wavelength_range, absorbance_max,
    fixed_pyruvate_mM,
    verbose
):
    """Process using spectral deconvolution methods with fixed pyruvate"""

    nadh_concentrations = []
    pyr_concentrations = []
    r_squared_values = []
    intercepts = []

    total_points = len(spectral_df)

    if verbose:
        print(f"Processing {total_points} time points with fixed pyruvate = {fixed_pyruvate_mM:.4f} mM...")

    for idx, row in spectral_df.iterrows():
        # Build spectrum dataframe
        spectrum_data = []
        for col in spectral_cols:
            try:
                wavelength = float(col)
                absorbance = row[col]
                spectrum_data.append({'Wavelength': wavelength, 'Absorbance': absorbance})
            except:
                continue

        spectrum_df_temp = pd.DataFrame(spectrum_data)

        # Set parameters based on method
        if method == 'full_spectrum':
            wl_range = None
            abs_max = None
        elif method == 'constrained':
            wl_range = wavelength_range
            abs_max = absorbance_max
        else:
            raise ValueError(f"Unknown method: {method}")

        # Calculate concentrations with FIXED PYRUVATE
        result = calculate_concentrations(
            spectrum_df=spectrum_df_temp,
            standards_df=standards_df,
            wavelength_range=wl_range,
            absorbance_max=abs_max,
            fit_intercept=True,
            fixed_pyr=fixed_pyruvate_mM,
            plot=False
        )

        nadh_concentrations.append(result['NADH_Conc'])
        pyr_concentrations.append(result['PYR_Conc'])
        r_squared_values.append(result['R_squared'])
        intercepts.append(result['Intercept'])

        # Progress indicator
        if verbose and (idx % 20 == 0 or idx == total_points - 1):
            print(f"  Processing: {idx+1}/{total_points} time points...", end='\r')

    if verbose:
        print()
        print()

    # Create results dataframe
    results_df = pd.DataFrame({
        'Time_s': spectral_df['Time_s'].values,
        'NADH_mM': nadh_concentrations,
        'Pyruvate_mM': pyr_concentrations,
        'R_squared': r_squared_values,
        'Intercept': intercepts
    })

    return results_df


def _print_summary_statistics(results_df, assay_start_time):
    """Print summary statistics for the processed data"""
    print("="*80)
    print("SUMMARY STATISTICS")
    print("="*80)

    # Blank pyruvate info
    if 'Blank_Pyruvate_mM' in results_df.columns and not pd.isna(results_df['Blank_Pyruvate_mM'].iloc[0]):
        print(f"\nFixed Pyruvate Concentration: {results_df['Blank_Pyruvate_mM'].iloc[0]:.4f} mM")
        print("  (Calculated from blank spectrum assuming NADH=0)")

    # Overall statistics
    print("\nOverall NADH Concentration:")
    print(f"  Mean: {results_df['NADH_mM'].mean():.6f} mM")
    print(f"  Std:  {results_df['NADH_mM'].std():.6f} mM")
    print(f"  Min:  {results_df['NADH_mM'].min():.6f} mM")
    print(f"  Max:  {results_df['NADH_mM'].max():.6f} mM")

    if 'R_squared' in results_df.columns and not results_df['R_squared'].isna().all():
        print("\nFit Quality (R²):")
        print(f"  Mean: {results_df['R_squared'].mean():.6f}")
        print(f"  Min:  {results_df['R_squared'].min():.6f}")
        poor_fits = (results_df['R_squared'] < 0.95).sum()
        if poor_fits > 0:
            print(f"  ⚠️  Warning: {poor_fits} time points with R² < 0.95")

    # Before/after assay start
    before_start = results_df[results_df['Time_s'] < assay_start_time]
    after_start = results_df[results_df['Time_s'] >= assay_start_time]

    if len(before_start) > 0:
        print(f"\nBefore Assay Start (t < {assay_start_time} s):")
        print(f"  NADH: {before_start['NADH_mM'].mean():.6f} ± {before_start['NADH_mM'].std():.6f} mM")

    if len(after_start) > 0:
        print(f"\nAfter Assay Start (t ≥ {assay_start_time} s):")
        print(f"  NADH: {after_start['NADH_mM'].mean():.6f} ± {after_start['NADH_mM'].std():.6f} mM")

        # Calculate rate if there are enough points
        if len(after_start) > 5:
            initial_phase = after_start[after_start['Time_Relative_s'] <= 30].head(10)
            if len(initial_phase) > 2:
                coeffs = np.polyfit(initial_phase['Time_Relative_s'],
                                   initial_phase['NADH_mM'], 1)
                initial_rate = coeffs[0]
                print(f"  Initial rate: {initial_rate:.6f} mM/s ({initial_rate*60:.6f} mM/min)")

    print()


def _generate_diagnostic_plots(
    results_df, spectral_df, spectral_cols, standards_df,
    assay_start_time, plot_title, method
):
    """Generate diagnostic plots for the time course"""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'NADH Concentration vs Time',
            'Fit Quality (R²) vs Time',
            'Intercept vs Time',
            'Sample Spectra at Key Time Points'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )

    # Panel 1: NADH concentration
    fig.add_trace(
        go.Scatter(
            x=results_df['Time_Relative_s'],
            y=results_df['NADH_mM'],
            mode='lines+markers',
            name='NADH',
            line=dict(color='royalblue', width=2),
            marker=dict(size=5)
        ),
        row=1, col=1
    )

    fig.add_vline(x=0, line_dash="dash", line_color="red",
                  annotation_text="Assay Start", row=1, col=1)

    # Panel 2: R² values
    if 'R_squared' in results_df.columns and not results_df['R_squared'].isna().all():
        fig.add_trace(
            go.Scatter(
                x=results_df['Time_Relative_s'],
                y=results_df['R_squared'],
                mode='lines+markers',
                name='R²',
                line=dict(color='green', width=2),
                marker=dict(size=5),
                showlegend=False
            ),
            row=1, col=2
        )

        fig.add_hline(y=0.95, line_dash="dash", line_color="gray",
                     annotation_text="R²=0.95", row=1, col=2)

    # Panel 3: Intercept
    if 'Intercept' in results_df.columns:
        fig.add_trace(
            go.Scatter(
                x=results_df['Time_Relative_s'],
                y=results_df['Intercept'],
                mode='lines+markers',
                name='Intercept',
                line=dict(color='gray', width=2),
                marker=dict(size=5),
                showlegend=False
            ),
            row=2, col=1
        )

    # Panel 4: Sample spectra
    n_spectra = min(4, len(spectral_df))
    indices = np.linspace(0, len(spectral_df)-1, n_spectra, dtype=int)
    colors = ['blue', 'green', 'orange', 'red']

    for i, idx in enumerate(indices):
        row = spectral_df.iloc[idx]
        wavelengths = []
        absorbances = []

        for col in spectral_cols:
            try:
                wl = float(col)
                if 250 <= wl <= 500:
                    wavelengths.append(wl)
                    absorbances.append(row[col])
            except:
                continue

        time_point = row['Time_s']
        relative_time = time_point - assay_start_time

        fig.add_trace(
            go.Scatter(
                x=wavelengths,
                y=absorbances,
                mode='lines',
                name=f't={relative_time:.0f}s',
                line=dict(color=colors[i], width=2)
            ),
            row=2, col=2
        )

    # Update axes
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=1, col=1)
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=1, col=2)
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=2, col=1)
    fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=2)

    fig.update_yaxes(title_text="[NADH] (mM)", row=1, col=1)
    fig.update_yaxes(title_text="R²", row=1, col=2)
    fig.update_yaxes(title_text="Intercept", row=2, col=1)
    fig.update_yaxes(title_text="Absorbance", row=2, col=2)

    # Layout
    if plot_title is None:
        blank_pyr = results_df['Blank_Pyruvate_mM'].iloc[0] if 'Blank_Pyruvate_mM' in results_df.columns else None
        if blank_pyr is not None and not pd.isna(blank_pyr):
            plot_title = f"PDC Time Course Analysis ({method} method, PYR={blank_pyr:.2f}mM)"
        else:
            plot_title = f"PDC Time Course Analysis ({method} method)"

    fig.update_layout(
        height=800,
        title_text=plot_title,
        template='plotly_white',
        showlegend=True,
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
    )

    fig.show()

"""

In [26]:
def process_pdc_timecourse(
    spectral_df,
    standards_df,
    assay_start_time,
    blank_time=None,
    initial_pyruvate_mM=None,
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    plot=False,
    plot_title=None,
    verbose=True
):
    """
    Process PDC (Pyruvate Dehydrogenase Complex) time course data to calculate NADH concentrations.

    This function processes spectrophotometric time course data from a PDC enzyme assay,
    calculating NADH concentrations at each time point using spectral deconvolution.

    **Pyruvate Concentration Logic:**
    - If blank_time is provided: Calculates pyruvate from blank spectrum (assuming NADH=0)
    - If blank_time is None: Must provide initial_pyruvate_mM manually
    - initial_pyruvate_mM is optional when blank_time is given (used only as fallback)

    Parameters
    ----------
    spectral_df : pandas.DataFrame
        DataFrame containing spectral time course data with:
        - 'Time_s' column: time in seconds
        - Wavelength columns: named with wavelength values (e.g., '340', '350', etc.)
        - Optional 'sample' column for identification

    standards_df : pandas.DataFrame
        DataFrame with extinction coefficient standards containing:
        - 'Wavelength' column: wavelength values
        - 'NADH_Coeff' column: NADH extinction coefficients (mM⁻¹cm⁻¹)
        - 'PYR_Coeff' column: Pyruvate extinction coefficients (mM⁻¹cm⁻¹)

    assay_start_time : float
        Time point (in seconds) when the assay was initiated (e.g., when substrate was added)

    blank_time : float, optional (default=None)
        Time point (in seconds) to use for blank/baseline correction
        If provided, pyruvate concentration is calculated from this spectrum (assuming NADH=0)
        If None, must provide initial_pyruvate_mM manually

    initial_pyruvate_mM : float, optional (default=None)
        Initial pyruvate concentration in mM
        - If blank_time is provided: Used only as fallback if calculation fails
        - If blank_time is None: REQUIRED - used directly for all calculations

    method : str, optional (default='constrained')
        Method for NADH calculation:
        - 'single_wavelength': Use Abs340 only with blank subtraction
        - 'full_spectrum': Use entire spectrum for deconvolution
        - 'constrained': Use wavelength_range and absorbance_max constraints (recommended)

    wavelength_range : tuple, optional (default=(320, 420))
        Tuple of (min_wavelength, max_wavelength) in nm
        Only used if method='constrained'

    absorbance_max : float, optional (default=2.5)
        Maximum absorbance threshold to exclude saturated data
        Only used if method='constrained'

    plot : bool, optional (default=False)
        If True, generates diagnostic plots

    plot_title : str, optional (default=None)
        Custom title for plots. Auto-generates if None.

    verbose : bool, optional (default=True)
        If True, prints progress and summary statistics

    Returns
    -------
    pandas.DataFrame
        DataFrame with the following columns:
        - 'Time_s': Time in seconds
        - 'Time_Relative_s': Time relative to assay_start_time (0 = assay start)
        - 'NADH_mM': NADH concentration in mM
        - 'Pyruvate_mM': Pyruvate concentration in mM (fixed value)
        - 'R_squared': Goodness of fit for each time point
        - 'Intercept': Baseline offset at each time point
        - 'Blank_Pyruvate_mM': Pyruvate concentration calculated from blank
        - 'Method': Method used for calculation
        - 'Fixed_Pyruvate': Whether pyruvate was fixed (boolean)

    Examples
    --------
    >>> # Recommended: Automatic pyruvate calculation from blank
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     blank_time=180.0,  # Pyruvate auto-calculated from this spectrum
    ...     plot=True
    ... )

    >>> # Manual pyruvate specification (no blank_time)
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     initial_pyruvate_mM=100.0,  # Must specify manually
    ...     plot=True
    ... )

    >>> # Provide fallback in case blank calculation fails
    >>> results = process_pdc_timecourse(
    ...     spectral_df=cell_1_df,
    ...     standards_df=standards_df,
    ...     assay_start_time=180.0,
    ...     blank_time=180.0,
    ...     initial_pyruvate_mM=100.0,  # Used only if blank calc fails
    ...     plot=True
    ... )

    Notes
    -----
    - The function assumes a 1 cm path length
    - When blank_time is provided, pyruvate is calculated assuming NADH=0
    - For PDC assays, NADH production indicates pyruvate consumption
    - R² values < 0.95 may indicate poor fit quality
    """

    # Validate parameters
    if blank_time is None and initial_pyruvate_mM is None:
        import warnings
        warnings.warn(
            "Neither blank_time nor initial_pyruvate_mM was specified. "
            "Setting initial_pyruvate_mM to 0. This assumes the spectrophotometer "
            "was blanked after pyruvate addition.",
            UserWarning
        )
        initial_pyruvate_mM = 0.0

    if verbose:
        print("="*80)
        print("PDC TIME COURSE PROCESSING")
        print("="*80)
        print(f"Method: {method}")
        print(f"Assay start time: {assay_start_time} s")
        if blank_time is not None:
            print(f"Blank time: {blank_time} s (pyruvate will be calculated)")
            if initial_pyruvate_mM is not None:
                print(f"Fallback pyruvate: {initial_pyruvate_mM} mM (if calculation fails)")
        else:
            print(f"Manual pyruvate: {initial_pyruvate_mM} mM (no blank calculation)")
        if method == 'constrained':
            print(f"Wavelength range: {wavelength_range[0]}-{wavelength_range[1]} nm")
            print(f"Max absorbance: {absorbance_max}")
        print()

    # Identify wavelength columns
    non_spectral_cols = ['sample', 'Time_s', 'filename', 'NADH_Conc_SingleWav',
                         'NADH_Conc_Spectral', 'NADH_Method1', 'NADH_Method2',
                         'NADH_Method3', 'NADH_Method4']
    spectral_cols = [c for c in spectral_df.columns if c not in non_spectral_cols]

    # Determine pyruvate concentration to use
    if blank_time is not None and method != 'single_wavelength':
        # Calculate pyruvate from blank spectrum
        blank_pyruvate_mM = _calculate_blank_pyruvate(
            spectral_df, standards_df, spectral_cols, blank_time,
            method, wavelength_range, absorbance_max,
            initial_pyruvate_mM, verbose
        )

        # If both blank_time and initial_pyruvate_mM were specified, compare them
        if initial_pyruvate_mM is not None:
            percent_difference = abs(blank_pyruvate_mM - initial_pyruvate_mM) / initial_pyruvate_mM * 100

            # Check if measured value is within 50% of specified value
            if percent_difference > 50:
                import warnings
                warnings.warn(
                    f"Measured pyruvate concentration is out of range!\n"
                    f"  Specified value: {initial_pyruvate_mM:.4f} mM\n"
                    f"  Measured value:  {blank_pyruvate_mM:.4f} mM\n"
                    f"  Percent difference: {percent_difference:.1f}%\n"
                    f"The measured value differs by more than 50% from the specified value.",
                    UserWarning
                )
    elif initial_pyruvate_mM is not None:
        # Use manually specified pyruvate
        blank_pyruvate_mM = initial_pyruvate_mM
        if verbose and method != 'single_wavelength':
            print(f"Using manually specified pyruvate: {blank_pyruvate_mM:.4f} mM")
            print()
    else:
        raise ValueError("This should not happen - parameter validation failed")

    # Process based on method
    if method == 'single_wavelength':
        if blank_time is None:
            raise ValueError("single_wavelength method requires blank_time to be specified")
        results_df = _process_single_wavelength(
            spectral_df, blank_time, verbose
        )
        results_df['Blank_Pyruvate_mM'] = np.nan  # Not applicable
    else:
        results_df = _process_spectral_deconvolution(
            spectral_df, standards_df, spectral_cols,
            method, wavelength_range, absorbance_max,
            blank_pyruvate_mM,
            verbose
        )
        results_df['Blank_Pyruvate_mM'] = blank_pyruvate_mM

    # Add relative time column (0 = assay start)
    results_df['Time_Relative_s'] = results_df['Time_s'] - assay_start_time

    # Add metadata columns
    results_df['Method'] = method
    results_df['Fixed_Pyruvate'] = True if method != 'single_wavelength' else False
    results_df['Assay_Start_s'] = assay_start_time
    results_df['Blank_Time_s'] = blank_time if blank_time is not None else np.nan

    # Reorder columns for clarity
    col_order = ['Time_s', 'Time_Relative_s', 'NADH_mM', 'Pyruvate_mM',
                 'R_squared', 'Intercept', 'Blank_Pyruvate_mM',
                 'Method', 'Fixed_Pyruvate', 'Assay_Start_s', 'Blank_Time_s']
    results_df = results_df[[c for c in col_order if c in results_df.columns]]

    # Print summary statistics
    if verbose:
        _print_summary_statistics(results_df, assay_start_time)

    # Generate plots if requested
    if plot:
        _generate_diagnostic_plots(
            results_df, spectral_df, spectral_cols, standards_df,
            assay_start_time, plot_title, method
        )

    return results_df


def _calculate_blank_pyruvate(
    spectral_df, standards_df, spectral_cols, blank_time,
    method, wavelength_range, absorbance_max,
    fallback_pyruvate_mM, verbose
):
    """
    Calculate pyruvate concentration at blank_time assuming NADH=0.

    Returns the calculated pyruvate or fallback value if calculation fails.
    """
    if verbose:
        print("Calculating pyruvate concentration at blank time...")
        print(f"  Assumption: NADH = 0 mM at t = {blank_time} s")

    # Find the row closest to blank_time
    idx_blank = (spectral_df['Time_s'] - blank_time).abs().idxmin()
    blank_row = spectral_df.iloc[idx_blank]
    actual_blank_time = blank_row['Time_s']

    # Build spectrum dataframe for blank time point
    spectrum_data = []
    for col in spectral_cols:
        try:
            wavelength = float(col)
            absorbance = blank_row[col]
            spectrum_data.append({'Wavelength': wavelength, 'Absorbance': absorbance})
        except:
            continue

    spectrum_df_blank = pd.DataFrame(spectrum_data)

    # Set parameters based on method
    if method == 'full_spectrum':
        wl_range = None
        abs_max = None
    elif method == 'constrained':
        wl_range = wavelength_range
        abs_max = absorbance_max
    else:
        raise ValueError(f"Unknown method: {method}")

    # Calculate concentrations with NADH fixed at 0
    try:
        result = calculate_concentrations(
            spectrum_df=spectrum_df_blank,
            standards_df=standards_df,
            wavelength_range=wl_range,
            absorbance_max=abs_max,
            fit_intercept=True,
            fixed_nadh=0.0,  # Fix NADH at 0 for blank
            plot=False
        )

        blank_pyruvate = result['PYR_Conc']
        r_squared = result['R_squared']

        if verbose:
            print(f"  Actual blank time: {actual_blank_time:.1f} s")
            print(f"  Calculated pyruvate: {blank_pyruvate:.4f} mM")
            print(f"  Fit R²: {r_squared:.6f}")

            # Show difference from fallback if provided
            if fallback_pyruvate_mM is not None:
                diff = blank_pyruvate - fallback_pyruvate_mM
                pct_diff = 100 * diff / fallback_pyruvate_mM
                print(f"  Difference from fallback: {diff:+.4f} mM ({pct_diff:+.2f}%)")

            if r_squared < 0.95:
                print(f"  ⚠️  Warning: Low R² ({r_squared:.4f}) at blank time")

            print()

        return blank_pyruvate

    except Exception as e:
        if verbose:
            print(f"  ⚠️  Error calculating blank pyruvate: {e}")
            if fallback_pyruvate_mM is not None:
                print(f"  Using fallback value: {fallback_pyruvate_mM:.4f} mM")
            else:
                print(f"  ERROR: No fallback value provided!")
            print()

        if fallback_pyruvate_mM is not None:
            return fallback_pyruvate_mM
        else:
            raise ValueError(
                "Failed to calculate pyruvate from blank and no fallback value provided. "
                "Please provide initial_pyruvate_mM as a fallback."
            )


def _process_single_wavelength(spectral_df, blank_time, verbose):
    """Process using single wavelength (340 nm) method"""
    col_340 = '340'

    if col_340 not in spectral_df.columns:
        raise ValueError(f"Column '{col_340}' not found in spectral_df")

    # Get blank value
    idx_blank = (spectral_df['Time_s'] - blank_time).abs().idxmin()
    blank_340 = spectral_df.loc[idx_blank, col_340]

    # Calculate NADH
    epsilon_340 = 6.22  # mM^-1 cm^-1
    nadh_conc = (spectral_df[col_340] - blank_340) / epsilon_340

    if verbose:
        print(f"Single wavelength method:")
        print(f"  Blank Abs340: {blank_340:.6f}")
        print(f"  Extinction coeff: {epsilon_340} mM⁻¹cm⁻¹")
        print()

    # Create results dataframe
    results_df = pd.DataFrame({
        'Time_s': spectral_df['Time_s'],
        'NADH_mM': nadh_conc,
        'Pyruvate_mM': np.nan,
        'R_squared': np.nan,
        'Intercept': -blank_340
    })

    return results_df


def _process_spectral_deconvolution(
    spectral_df, standards_df, spectral_cols,
    method, wavelength_range, absorbance_max,
    fixed_pyruvate_mM,
    verbose
):
    """Process using spectral deconvolution methods with fixed pyruvate"""

    nadh_concentrations = []
    pyr_concentrations = []
    r_squared_values = []
    intercepts = []

    total_points = len(spectral_df)

    if verbose:
        print(f"Processing {total_points} time points with fixed pyruvate = {fixed_pyruvate_mM:.4f} mM...")

    for idx, row in spectral_df.iterrows():
        # Build spectrum dataframe
        spectrum_data = []
        for col in spectral_cols:
            try:
                wavelength = float(col)
                absorbance = row[col]
                spectrum_data.append({'Wavelength': wavelength, 'Absorbance': absorbance})
            except:
                continue

        spectrum_df_temp = pd.DataFrame(spectrum_data)

        # Set parameters based on method
        if method == 'full_spectrum':
            wl_range = None
            abs_max = None
        elif method == 'constrained':
            wl_range = wavelength_range
            abs_max = absorbance_max
        else:
            raise ValueError(f"Unknown method: {method}")

        # Calculate concentrations with FIXED PYRUVATE
        result = calculate_concentrations(
            spectrum_df=spectrum_df_temp,
            standards_df=standards_df,
            wavelength_range=wl_range,
            absorbance_max=abs_max,
            fit_intercept=True,
            fixed_pyr=fixed_pyruvate_mM,
            plot=False
        )

        nadh_concentrations.append(result['NADH_Conc'])
        pyr_concentrations.append(result['PYR_Conc'])
        r_squared_values.append(result['R_squared'])
        intercepts.append(result['Intercept'])

        # Progress indicator
        if verbose and (idx % 20 == 0 or idx == total_points - 1):
            print(f"  Processing: {idx+1}/{total_points} time points...", end='\r')

    if verbose:
        print()
        print()

    # Create results dataframe
    results_df = pd.DataFrame({
        'Time_s': spectral_df['Time_s'].values,
        'NADH_mM': nadh_concentrations,
        'Pyruvate_mM': pyr_concentrations,
        'R_squared': r_squared_values,
        'Intercept': intercepts
    })

    return results_df


def _print_summary_statistics(results_df, assay_start_time):
    """Print summary statistics for the processed data"""
    print("="*80)
    print("SUMMARY STATISTICS")
    print("="*80)

    # Blank pyruvate info
    if 'Blank_Pyruvate_mM' in results_df.columns and not pd.isna(results_df['Blank_Pyruvate_mM'].iloc[0]):
        print(f"\nFixed Pyruvate Concentration: {results_df['Blank_Pyruvate_mM'].iloc[0]:.4f} mM")
        print("  (Calculated from blank spectrum assuming NADH=0)")

    # Overall statistics
    print("\nOverall NADH Concentration:")
    print(f"  Mean: {results_df['NADH_mM'].mean():.6f} mM")
    print(f"  Std:  {results_df['NADH_mM'].std():.6f} mM")
    print(f"  Min:  {results_df['NADH_mM'].min():.6f} mM")
    print(f"  Max:  {results_df['NADH_mM'].max():.6f} mM")

    if 'R_squared' in results_df.columns and not results_df['R_squared'].isna().all():
        print("\nFit Quality (R²):")
        print(f"  Mean: {results_df['R_squared'].mean():.6f}")
        print(f"  Min:  {results_df['R_squared'].min():.6f}")
        poor_fits = (results_df['R_squared'] < 0.95).sum()
        if poor_fits > 0:
            print(f"  ⚠️  Warning: {poor_fits} time points with R² < 0.95")

    # Before/after assay start
    before_start = results_df[results_df['Time_s'] < assay_start_time]
    after_start = results_df[results_df['Time_s'] >= assay_start_time]

    if len(before_start) > 0:
        print(f"\nBefore Assay Start (t < {assay_start_time} s):")
        print(f"  NADH: {before_start['NADH_mM'].mean():.6f} ± {before_start['NADH_mM'].std():.6f} mM")

    if len(after_start) > 0:
        print(f"\nAfter Assay Start (t ≥ {assay_start_time} s):")
        print(f"  NADH: {after_start['NADH_mM'].mean():.6f} ± {after_start['NADH_mM'].std():.6f} mM")

        # Calculate rate if there are enough points
        if len(after_start) > 5:
            initial_phase = after_start[after_start['Time_Relative_s'] <= 30].head(10)
            if len(initial_phase) > 2:
                coeffs = np.polyfit(initial_phase['Time_Relative_s'],
                                   initial_phase['NADH_mM'], 1)
                initial_rate = coeffs[0]
                print(f"  Initial rate: {initial_rate:.6f} mM/s ({initial_rate*60:.6f} mM/min)")

    print()


def _generate_diagnostic_plots(
    results_df, spectral_df, spectral_cols, standards_df,
    assay_start_time, plot_title, method
):
    """Generate diagnostic plots for the time course"""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'NADH Concentration vs Time',
            'Fit Quality (R²) vs Time',
            'Intercept vs Time',
            'Sample Spectra at Key Time Points'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )

    # Panel 1: NADH concentration
    fig.add_trace(
        go.Scatter(
            x=results_df['Time_Relative_s'],
            y=results_df['NADH_mM'],
            mode='lines+markers',
            name='NADH',
            line=dict(color='royalblue', width=2),
            marker=dict(size=5)
        ),
        row=1, col=1
    )

    fig.add_vline(x=0, line_dash="dash", line_color="red",
                  annotation_text="Assay Start", row=1, col=1)

    # Panel 2: R² values
    if 'R_squared' in results_df.columns and not results_df['R_squared'].isna().all():
        fig.add_trace(
            go.Scatter(
                x=results_df['Time_Relative_s'],
                y=results_df['R_squared'],
                mode='lines+markers',
                name='R²',
                line=dict(color='green', width=2),
                marker=dict(size=5),
                showlegend=False
            ),
            row=1, col=2
        )

        fig.add_hline(y=0.95, line_dash="dash", line_color="gray",
                     annotation_text="R²=0.95", row=1, col=2)

    # Panel 3: Intercept
    if 'Intercept' in results_df.columns:
        fig.add_trace(
            go.Scatter(
                x=results_df['Time_Relative_s'],
                y=results_df['Intercept'],
                mode='lines+markers',
                name='Intercept',
                line=dict(color='gray', width=2),
                marker=dict(size=5),
                showlegend=False
            ),
            row=2, col=1
        )

    # Panel 4: Sample spectra
    n_spectra = min(4, len(spectral_df))
    indices = np.linspace(0, len(spectral_df)-1, n_spectra, dtype=int)
    colors = ['blue', 'green', 'orange', 'red']

    for i, idx in enumerate(indices):
        row = spectral_df.iloc[idx]
        wavelengths = []
        absorbances = []

        for col in spectral_cols:
            try:
                wl = float(col)
                if 250 <= wl <= 500:
                    wavelengths.append(wl)
                    absorbances.append(row[col])
            except:
                continue

        time_point = row['Time_s']
        relative_time = time_point - assay_start_time

        fig.add_trace(
            go.Scatter(
                x=wavelengths,
                y=absorbances,
                mode='lines',
                name=f't={relative_time:.0f}s',
                line=dict(color=colors[i], width=2)
            ),
            row=2, col=2
        )

    # Update axes
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=1, col=1)
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=1, col=2)
    fig.update_xaxes(title_text="Time from Assay Start (s)", row=2, col=1)
    fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=2)

    fig.update_yaxes(title_text="[NADH] (mM)", row=1, col=1)
    fig.update_yaxes(title_text="R²", row=1, col=2)
    fig.update_yaxes(title_text="Intercept", row=2, col=1)
    fig.update_yaxes(title_text="Absorbance", row=2, col=2)

    # Layout
    if plot_title is None:
        blank_pyr = results_df['Blank_Pyruvate_mM'].iloc[0] if 'Blank_Pyruvate_mM' in results_df.columns else None
        if blank_pyr is not None and not pd.isna(blank_pyr):
            plot_title = f"PDC Time Course Analysis ({method} method, PYR={blank_pyr:.2f}mM)"
        else:
            plot_title = f"PDC Time Course Analysis ({method} method)"

    fig.update_layout(
        height=800,
        title_text=plot_title,
        template='plotly_white',
        showlegend=True,
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
    )

    fig.show()

In [29]:
# Define the full file path provided in the instructions
file_path = '/content/drive/MyDrive/Research/PDC+ADH+FDH assay data Evelyn 2025/KD files from Agilent spec/0113 1600M PYR PDC-1.csv'

# Load the kinetic data file
kinetic_df = pd.read_csv(file_path)

# Filter for CELL_1 samples
cell_1_df = kinetic_df[kinetic_df['sample'] == 'CELL_1'].copy()

process_pdc_timecourse(spectral_df = cell_1_df,
                       standards_df = standards_df,
                       assay_start_time = 390,
                       blank_time = 40,
                       absorbance_max=2,
                       initial_pyruvate_mM = 1600,
                       #method = 'constrained',
                       plot = True
                       )

PDC TIME COURSE PROCESSING
Method: constrained
Assay start time: 390 s
Blank time: 40 s (pyruvate will be calculated)
Fallback pyruvate: 1600 mM (if calculation fails)
Wavelength range: 320-420 nm
Max absorbance: 2

Calculating pyruvate concentration at blank time...
  Assumption: NADH = 0 mM at t = 40 s
  Actual blank time: 39.1 s
  Calculated pyruvate: 1359.8637 mM
  Fit R²: 0.993713
  Difference from fallback: -240.1363 mM (-15.01%)

Processing 118 time points with fixed pyruvate = 1359.8637 mM...
  Processing: 118/118 time points...

SUMMARY STATISTICS

Fixed Pyruvate Concentration: 1359.8637 mM
  (Calculated from blank spectrum assuming NADH=0)

Overall NADH Concentration:
  Mean: 0.707925 mM
  Std:  0.588852 mM
  Min:  0.016270 mM
  Max:  1.353285 mM

Fit Quality (R²):
  Mean: 0.680696
  Min:  0.035414
  ⚠️  Warning: 50 time points with R² < 0.95

Before Assay Start (t < 390 s):
  NADH: 1.023096 ± 0.536828 mM

After Assay Start (t ≥ 390 s):
  NADH: 0.370636 ± 0.437821 mM
  Initia

,Time_s,Time_Relative_s,NADH_mM,Pyruvate_mM,R_squared,Intercept,Blank_Pyruvate_mM,Method,Fixed_Pyruvate,Assay_Start_s,Blank_Time_s
0,1.2,-388.8,0.020509,1359.863664,0.087893,-0.250149,1359.863664,constrained,True,390,40
1,7.1,-382.9,0.019997,1359.863664,0.084183,-0.250164,1359.863664,constrained,True,390,40
2,13.4,-376.6,0.020171,1359.863664,0.084815,-0.250237,1359.863664,constrained,True,390,40
3,19.9,-370.1,0.019933,1359.863664,0.079706,-0.250087,1359.863664,constrained,True,390,40
4,26.3,-363.7,0.019359,1359.863664,0.076067,-0.249953,1359.863664,constrained,True,390,40
...,...,...,...,...,...,...,...,...,...,...,...
113,723.9,333.9,0.016270,1359.863664,0.035414,-0.244428,1359.863664,constrained,True,390,40
114,730.3,340.3,0.016655,1359.863664,0.038336,-0.244195,1359.863664,constrained,True,390,40
115,736.6,346.6,0.016800,1359.863664,0.038517,-0.244866,1359.863664,constrained,True,390,40
116,743.1,353.1,0.017622,1359.863664,0.043679,-0.244488,1359.863664,constrained,True,390,40
